In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [2]:
pokemon = pd.DataFrame(columns = ["number", "name", "type", "ability", "hidden_ability", 
                                 "gender_ratio", "hatch_cycle","egg_group", "Base_Experience_Yield", "height", "weight",
                                 "leveling_rate", "color", "base_friendship", "category"], index = range(1, 1026))
pokemon

,number,name,type,ability,hidden_ability,gender_ratio,hatch_cycle,egg_group,Base_Experience_Yield,height,weight,leveling_rate,color,base_friendship,category
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
pokemon = pd.read_csv("/home/user/Desktop/Data_analysis_projects/Pokemon/data.csv", index_col = 0)

In [20]:
# Creating a function for scraping pokemon data

def scrap_pokemon_details(pokemon_name):
    """
    scrape details of pokemon from bulbapedia

    Args :
        pokemon_name(str) name of the pokemon, eg : Greninja 

    Returns:
        A dictionary will all the details 
    """
    
    try: 
        # Give URL 
        url = f"https://bulbapedia.bulbagarden.net/wiki/{pokemon_name}_(Pok%C3%A9mon)"
        
        # fetching the webpage 
        response = requests.get(url)
        response.raise_for_status()
        text = response.text
        
        # Creating a BeautifulSoup instance 
        soup = BeautifulSoup(text, "lxml")
        
        # Overall data 
        data = soup.find_all("td", class_ = "roundy")
        
        if len(data) < 40 :
            print(f"Don't have enought data for {pokemon_data}")
            return None
        
        # name, number, category
        basic = data[0]
        name = basic.find(["big", "b"]).text.strip()
        category = basic.find(["span"]).text.strip()
        number = soup.find(["th","span", "big"], class_ = "roundy").text.replace("\n", "").strip()
        
        # Type 
        type_data = data[1]
        type_data_2 =  type_data.find_all(["span", "a"])
        if type_data_2[4].text.strip() != "Unknown":
            type_ = f"{type_data_2[2].text.strip()} / {type_data_2[4].text.strip()}"
        else:
            type_ = type_data_2[2].text.strip()
        
        # ability and hidden ability
        ability_data = data[2]
        ability_data2 = ability_data.find_all(["a", "span"])
        clear_ability = []
        junk_text = ["Cacophony", "Abilities", "Gen IV+", "Battle Bond", "Battle Bond Greninja", "Gen VI+", "Gen VII+", "Ability", "Gen VIII+", "Gen IX+"]

        for i in ability_data2:
            text = i.text.strip()
            if text and text not in junk_text:
                clear_ability.append(text)
        
        if len(clear_ability) == 6:
            ability = f"{clear_ability[0]} / {clear_ability[2]}"
            hidden_ability = clear_ability[4]
        elif len(clear_ability) == 4:
            ability = clear_ability[0]
            hidden_ability = clear_ability[2]
        elif len(clear_ability) == 2:
            ability = clear_ability[0]
            hidden_ability = None
        else:
            ability = None
            hidden_ability = None
            
        # gender ratio
        gender_data = data[3]
        gender_data_2 = gender_data.find_all(["a"])
        gender_ratio = gender_data_2[1].text.strip()
        
        # Catch rate 
        catch_rate = data[4].find("small").text.strip()
        
        # egg group and hatch cycle 
        egg_group_data = data[5]
        egg_group = egg_group_data.find("table", class_ = "roundy").text.strip().replace("Gen VIII+", "").replace("\xa0", "").replace("or No Eggs DiscoveredCosplay/Cap", "")
        hatch_cycle_data = data[5].find_all("table", class_ = "roundy")
        hatch_cycle = hatch_cycle_data[1].text.strip().replace("\xa0", "").replace("Egg not obtainable", "")
        
        # Height and Weight 
        height_data = data[6].find_all("tr")
        height_data_2 = height_data[0].find_all("td")
        height = height_data_2[1].text.replace("\n", "").strip().replace(" m", "")
        height = float(height)
        weight_data = data[7].find_all("tr")
        weight_data_2 = weight_data[0].find_all("td")
        weight = weight_data_2[1].text.replace("\n", "").strip().replace(" kg", "")
        weight = float(weight)
        
        # Base Experience Yield 
        bey_data = data[9].find("tr")
        bey_data_2 = bey_data.find_all("td")
        bey = bey_data_2[2].text.replace("V+", "").replace("\n", "").strip()
        
        # Other details 
        leveling_rate = data[10].tr.text.strip()
        color = data[38].td.text.strip().replace("Other forms may have other colors.", "")
        base_friendship = data[39].tr.text.strip()
        
        return {"number" : number, "name" : name, "type" : type_, "ability" : ability, "hidden_ability" : hidden_ability, 
                 "gender_ratio" : gender_ratio, "hatch_cycle" : hatch_cycle, "egg_group" : egg_group, "Base_Experience_Yield" : bey, "leveling_rate" : leveling_rate,
                 "color" : color, "base_friendship" : base_friendship, "height" : height, "weight" : weight, "category" : category, "gen": 0}
            
    except requests.RequestException as e:
        print(f"Failed to fetch data for {pokemon_name}: {e}")
        return None
    except (AttributeError, IndexError) as e:
        print(f"Failed to parse data for {pokemon_name}: {e}")
        return None

            

In [242]:
# Bulbasaur evolution line
bulbasaur_data = scrap_pokemon_details("Bulbasaur")
pokemon.loc[1] = bulbasaur_data
ivysaur_data = scrap_pokemon_details("Ivysaur")
pokemon.loc[2] = ivysaur_data
venusaur_data = scrap_pokemon_details("Venusaur")
pokemon.loc[3] = venusaur_data

In [180]:
# Charmander line 
charmander_data = scrap_pokemon_details("Charmander")
pokemon.loc[4] = charmander_data
charmeleon_data = scrap_pokemon_details("Charmeleon")
pokemon.loc[5] = charmeleon_data
charizard_data = scrap_pokemon_details("Charizard")
charizard_data["ability"] = "Blaze"
charizard_data["hidden_ability"] = "Solar Power"
pokemon.loc[6] = charizard_data

In [181]:
# Squirtle line 
squirtle_data = scrap_pokemon_details("Squirtle")
pokemon.loc[7] = squirtle_data
wartortle_data = scrap_pokemon_details("Wartortle")
pokemon.loc[8] = wartortle_data
blastoise_data = scrap_pokemon_details("Blastoise")
pokemon.loc[9] = blastoise_data

In [182]:
# Caterpie line 
caterpie_data = scrap_pokemon_details("Caterpie")
pokemon.loc[10] = caterpie_data
metapod_data = scrap_pokemon_details("Metapod")
pokemon.loc[11] = metapod_data
butterfree_data = scrap_pokemon_details("Butterfree")
pokemon.loc[12] = butterfree_data

In [183]:
# Weedle line 
weedle_data = scrap_pokemon_details("Weedle")
pokemon.loc[13] = weedle_data
kakuna_data = scrap_pokemon_details("Kakuna")
pokemon.loc[14] = kakuna_data
beedrill_data = scrap_pokemon_details("Beedrill")
pokemon.loc[15] = beedrill_data

In [184]:
#pidgey line
pidgey_data = scrap_pokemon_details("Pidgey")
pokemon.loc[16] = pidgey_data
pidgeotto_data = scrap_pokemon_details("Pidgeotto")
pokemon.loc[17] = pidgeotto_data
pidgeot_data = scrap_pokemon_details("Pidgeot")
pidgeot_data["ability"] = "Keen Eye / Tangled Feet"
pidgeot_data["hidden_ability"] = "Big Pecks"
pokemon.loc[18] = pidgeot_data

In [185]:
# Rattata line
rattata_data = scrap_pokemon_details("Rattata")
rattata_data["ability"] = "Run Away / Guts"
rattata_data["hidden_ability"] = "Hustle"
pokemon.loc[19] = rattata_data
raticate_data = scrap_pokemon_details("Raticate")
raticate_data["ability"] = "Run Away / Guts"
raticate_data["hidden_ability"] = "Hustle"
pokemon.loc[20] = raticate_data

In [186]:
#Spearow line
spearow_data = scrap_pokemon_details("Spearow")
pokemon.loc[21] = spearow_data
fearow_data = scrap_pokemon_details("Fearow")
pokemon.loc[22] = fearow_data

In [187]:
# Ekans line
ekans_data = scrap_pokemon_details("Ekans")
pokemon.loc[23] = ekans_data
arbok_data = scrap_pokemon_details("Arbok")
pokemon.loc[24] = arbok_data

In [54]:
#Pikachu line
pikachu_data = scrap_pokemon_details("Pikachu")
pokemon.loc[25] = pikachu_data
raichu_data = scrap_pokemon_details("Raichu")
raichu_data["ability"] = "Static"
pokemon.loc[26] = raichu_data


In [189]:
#Sandshrew line
sandshrew_data = scrap_pokemon_details("Sandshrew")
sandshrew_data["ability"] = "Sand Veil"
sandshrew_data["hidden_ability"] = "Sand Rush"
pokemon.loc[27] = sandshrew_data
sandslash_data = scrap_pokemon_details("Sandslash")
sandslash_data["ability"] = "Sand Veil"
sandslash_data["hidden_ability"] = "Sand Rush"
pokemon.loc[28] = sandslash_data

In [26]:
# nidoran female line
nidoranf_data = scrap_pokemon_details("Nidoran♀")
pokemon.loc[29] = nidoranf_data
nidorina_data = scrap_pokemon_details("Nidorina")
pokemon.loc[30] = nidorina_data
nidoqueen_data = scrap_pokemon_details("Nidoqueen")
pokemon.loc[31] = nidoqueen_data

In [191]:
# Nidoran male line
nidoranm_data = scrap_pokemon_details("Nidoran♂")
pokemon.loc[32] = nidoranm_data
nidorino_data = scrap_pokemon_details("Nidorino")
pokemon.loc[33] = nidorino_data
nidoking_data = scrap_pokemon_details("Nidoking")
pokemon.loc[34] = nidoking_data

In [192]:
#Clefairy line 
clefairy_data = scrap_pokemon_details("Clefairy")
pokemon.loc[35] = clefairy_data
clefable_data = scrap_pokemon_details("Clefable")
pokemon.loc[36] = clefable_data

In [193]:
# Vulpix line
vulpix_data = scrap_pokemon_details("Vulpix")
vulpix_data["ability"] = "Flash Fire"
vulpix_data["hidden_ability"] = "Drought"
pokemon.loc[37] = vulpix_data
ninetails_data = scrap_pokemon_details("Ninetales")
ninetails_data["ability"] = "Flash Fire"
ninetails_data["hidden_ability"] = "Drought"
pokemon.loc[38] = ninetails_data

In [194]:
# Jigglypuff line
jigglypuff_data = scrap_pokemon_details("Jigglypuff")
pokemon.loc[39] = jigglypuff_data
wigglytuff_data = scrap_pokemon_details("Wigglytuff")
pokemon.loc[40] = wigglytuff_data

In [195]:
#Zubat line
zubat_data = scrap_pokemon_details("Zubat")
pokemon.loc[41] = zubat_data
golbat_data = scrap_pokemon_details("Golbat")
pokemon.loc[42] = golbat_data

In [196]:
#Oddish line
oddish_data = scrap_pokemon_details("Oddish")
pokemon.loc[43] = oddish_data
gloom_data = scrap_pokemon_details("Gloom")
pokemon.loc[44] = gloom_data
vileplume_data = scrap_pokemon_details("Vileplume")
pokemon.loc[45] = vileplume_data

In [197]:
# paras line
paras_data = scrap_pokemon_details("Paras")
pokemon.loc[46] = paras_data
parasect_data = scrap_pokemon_details("Parasect")
pokemon.loc[47] = parasect_data

In [198]:
# Venonat line
venonat_data = scrap_pokemon_details("Venonat")
pokemon.loc[48] = venonat_data
venomoth_data = scrap_pokemon_details("Venomoth")
pokemon.loc[49] = venomoth_data

In [199]:
# diglett line
diglett_data = scrap_pokemon_details("Diglett")
diglett_data["ability"] = "Sand Veil / Arena Trap"
diglett_data["hidden_ability"] = "Sand Force"
pokemon.loc[50] = diglett_data
dugtrio_data = scrap_pokemon_details("Dugtrio")
dugtrio_data["ability"] = "Sand Veil / Arena Trap"
dugtrio_data["hidden_ability"] = "Sand Force"
pokemon.loc[51] = dugtrio_data

In [22]:
# meowth line
meowth_data = scrap_pokemon_details("Meowth")
meowth_data["ability"] = "Pickup / Technician"
meowth_data["hidden_ability"] = "Unnerve"
pokemon.loc[52] = meowth_data
persian_data = scrap_pokemon_details("Persian")
persian_data["ability"] = "Limber / Technician"
persian_data["hidden_ability"] = "Unnerve"
pokemon.loc[53] = persian_data 

In [201]:
# Psyduck line
psyduck_data = scrap_pokemon_details("Psyduck")
pokemon.loc[54] = psyduck_data
golduck_data = scrap_pokemon_details("Golduck")
pokemon.loc[55] = golduck_data

In [202]:
# Mankey line 
mankey_data = scrap_pokemon_details("Mankey")
pokemon.loc[56] = mankey_data
primeape_data = scrap_pokemon_details("Primeape")
pokemon.loc[57] = primeape_data

In [32]:
# Growlithe line
growlithe_data = scrap_pokemon_details("Growlithe")
growlithe_data["ability"] = "Intimidate / Flash Fire"
growlithe_data["hidden_ability"] = "Justified"
growlithe_data["category"] = "Puppy Pokemon"
pokemon.loc[58] = growlithe_data
arcanine_data = scrap_pokemon_details("Arcanine")
arcanine_data["ability"] = "Intimidate / Flash Fire"
arcanine_data["hidden_ability"] = "Justified"
pokemon.loc[59] = arcanine_data

In [204]:
# poliwag line
poliwag_data = scrap_pokemon_details("Poliwag")
pokemon.loc[60] = poliwag_data
poliwhirl_data = scrap_pokemon_details("Poliwhirl")
pokemon.loc[61] = poliwhirl_data
poliwrath_data = scrap_pokemon_details("Poliwrath")
pokemon.loc[62] = poliwrath_data

In [205]:
# Abra line 
abra_data = scrap_pokemon_details("Abra")
pokemon.loc[63] = abra_data
kadabra_data = scrap_pokemon_details("Kadabra")
pokemon.loc[64] = kadabra_data
alakazam_data = scrap_pokemon_details("Alakazam")
alakazam_data["ability"] = "Synchronize / Inner Focus"
alakazam_data["hidden_ability"] = "Magic Guard"
pokemon.loc[65] = alakazam_data

In [206]:
# Machop line
machop_data = scrap_pokemon_details("Machop")
pokemon.loc[66] = machop_data
machoke_data = scrap_pokemon_details("Machoke")
pokemon.loc[67] = machoke_data
machamp_data = scrap_pokemon_details("Machamp")
pokemon.loc[68] = machamp_data

In [207]:
# bellsprout line 
bellsprout_data = scrap_pokemon_details("Bellsprout")
pokemon.loc[69] = bellsprout_data
weepinbell_data = scrap_pokemon_details("Weepinbell")
pokemon.loc[70] = weepinbell_data
victreebel_data = scrap_pokemon_details("Victreebel")
pokemon.loc[71] = victreebel_data

In [208]:
# tentacool line
tentacool_data = scrap_pokemon_details("Tentacool")
pokemon.loc[72] = tentacool_data
tentacruel_data = scrap_pokemon_details("Tentacruel")
pokemon.loc[73] = tentacruel_data

In [209]:
# geodude line 
geodude_data = scrap_pokemon_details("Geodude")
geodude_data["ability"] = "Rock Head / Sturdy"
geodude_data["hidden_ability"] = "Sand Veil"
pokemon.loc[74] = geodude_data
graveler_data = scrap_pokemon_details("Graveler")
graveler_data["ability"] = "Rock Head / Sturdy"
graveler_data["hidden_ability"] = "Sand Veil"
pokemon.loc[75] = graveler_data
golem_data = scrap_pokemon_details("Golem")
golem_data["ability"] = "Rock Head / Sturdy"
golem_data["hidden_ability"] = "Sand Veil"
pokemon.loc[76] = golem_data

In [31]:
# ponyta line 
ponyta_data = scrap_pokemon_details("Ponyta")
ponyta_data["ability"] = "Run Away / Flash Fire"
ponyta_data["hidden_ability"] = "Flame Body"
ponyta_data["category"] = "Fire Horse Pokémon"
pokemon.loc[77] = ponyta_data
rapidash_data = scrap_pokemon_details("Rapidash")
rapidash_data["ability"] = "Run Away / Flash Fire"
rapidash_data["hidden_ability"] = "Flame Body"
rapidash_data["category"] = "Fire Horse Pokémon"
pokemon.loc[78] = rapidash_data

In [211]:
# Slowpoke line 
slowpoke_data = scrap_pokemon_details("Slowpoke")
slowpoke_data["ability"] = "Oblivious / Own Tempo"
slowpoke_data["hidden_ability"] = "Regenerator"
pokemon.loc[79] = slowpoke_data
slowbro_data = scrap_pokemon_details("Slowbro")
slowbro_data["ability"] = "Oblivious / Own Tempo"
slowbro_data["hidden_ability"] = "Regenerator"
pokemon.loc[80] = slowbro_data

In [212]:
# magnemite line
magnemite_data = scrap_pokemon_details("Magnemite")
pokemon.loc[81] = magnemite_data
magneton_data = scrap_pokemon_details("Magneton")
pokemon.loc[82] = magneton_data

In [213]:
# Farfetch'd
farfetch_data = scrap_pokemon_details("Farfetch'd")
farfetch_data["ability"] = "Keen Eye / Inner Focus"
farfetch_data["hidden_ability"] = "Defiant"
pokemon.loc[83] = farfetch_data

In [214]:
# doduo line
doduo_data = scrap_pokemon_details("Doduo")
pokemon.loc[84] = doduo_data
dodrio_data = scrap_pokemon_details("Dodrio")
pokemon.loc[85] = dodrio_data

In [215]:
# Seel line 
seel_data = scrap_pokemon_details("Seel")
pokemon.loc[86] = seel_data
dewgong_data = scrap_pokemon_details("Dewgong")
pokemon.loc[87] = dewgong_data

In [216]:
# grimer line 
grimer_data = scrap_pokemon_details("Grimer")
grimer_data["ability"] = "Stench / Sticky Hold"
grimer_data["hidden_ability"] = "Poison Touch"
pokemon.loc[88] = grimer_data
muk_data = scrap_pokemon_details("Muk")
muk_data["ability"] = "Stench / Sticky Hold"
muk_data["hidden_ability"] = "Poison Touch"
pokemon.loc[89] = muk_data

In [217]:
# Shellder line 
shellder_data = scrap_pokemon_details("Shellder")
pokemon.loc[90] = shellder_data
cloyster_data = scrap_pokemon_details("Cloyster")
pokemon.loc[91] = cloyster_data

In [218]:
# Gastly line
gastly_data = scrap_pokemon_details("Gastly")
pokemon.loc[92] = gastly_data
haunter_data = scrap_pokemon_details("Haunter")
pokemon.loc[93] = haunter_data
gengar_data = scrap_pokemon_details("Gengar")
gengar_data["ability"] = "Cursed Body"
pokemon.loc[94] = gengar_data

In [219]:
# onix line 
onix_data = scrap_pokemon_details("Onix")
pokemon.loc[95] = onix_data

In [220]:
# drowzee line 
drowzee_data = scrap_pokemon_details("Drowzee")
pokemon.loc[96] = drowzee_data
hypno_data = scrap_pokemon_details("Hypno")
pokemon.loc[97] = hypno_data

In [221]:
# Krabby line 
krabby_data = scrap_pokemon_details("Krabby")
pokemon.loc[98] = krabby_data
kingler_data = scrap_pokemon_details("Kingler")
pokemon.loc[99] = kingler_data

In [30]:
# voltorb line 
voltorb_data = scrap_pokemon_details("Voltorb")
voltorb_data["category"] = "Ball Pokemon"
pokemon.loc[100] = voltorb_data
electrode_data = scrap_pokemon_details("Electrode")
electrode_data["category"] = "Ball Pokemon"
pokemon.loc[101] = electrode_data

In [223]:
# Exeggcute line 
exeggcute_data = scrap_pokemon_details("Exeggcute")
pokemon.loc[102] = exeggcute_data
exeggutor_data = scrap_pokemon_details("Exeggutor")
pokemon.loc[103] = exeggutor_data

In [224]:
# cubone line 
cubone_data = scrap_pokemon_details("Cubone")
pokemon.loc[104] = cubone_data
marowak_data = scrap_pokemon_details("Marowak")
marowak_data["ability"] = "Rock Head / Lightning Rod"
marowak_data["hidden_ability"] = "Battle Armor"
pokemon.loc[105] = marowak_data

In [225]:
# hitmonlee line  and lickitung
hitmonlee_data = scrap_pokemon_details("Hitmonlee")
pokemon.loc[106] = hitmonlee_data
hitmonchan_data = scrap_pokemon_details("Hitmonchan")
pokemon.loc[107] = hitmonchan_data
lickitung_data = scrap_pokemon_details("Lickitung")
pokemon.loc[108] = lickitung_data

In [226]:
# koffing
koffing_data = scrap_pokemon_details("Koffing")
koffing_data["ability"] = "Levitate / Neutralizing Gas"
koffing_data["hidden_ability"] = "Stench"
pokemon.loc[109] = koffing_data
weezing_data = scrap_pokemon_details("Weezing")
weezing_data["ability"] = "Levitate / Neutralizing Gas"
weezing_data["hidden_ability"] = "Stench"
pokemon.loc[110] = weezing_data

In [227]:
# Rhyhorn line
rhyhorn_data = scrap_pokemon_details("Rhyhorn")
pokemon.loc[111] = rhyhorn_data
rhydon_data = scrap_pokemon_details("Rhydon")
pokemon.loc[112] = rhydon_data

In [228]:
# Chansey line and tangela line
chansey_data = scrap_pokemon_details("Chansey")
pokemon.loc[113] = chansey_data
tangela_data = scrap_pokemon_details("Tangela")
pokemon.loc[114] = tangela_data

In [229]:
#Kangaskhan line
kangaskhan_data = scrap_pokemon_details("Kangaskhan")
kangaskhan_data["ability"] = "Early Bird / Scrappy"
kangaskhan_data["hidden_ability"] = "Inner Focus"
pokemon.loc[115] = kangaskhan_data

In [230]:
# horsea line 
horsea_data = scrap_pokemon_details("Horsea")
pokemon.loc[116] = horsea_data
seadra_data = scrap_pokemon_details("Seadra")
pokemon.loc[117] = seadra_data

In [231]:
# goldeen and staryu line
goldeen_data = scrap_pokemon_details("Goldeen")
pokemon.loc[118] = goldeen_data
seaking_data = scrap_pokemon_details("Seaking")
pokemon.loc[119] = seaking_data
staryu_data = scrap_pokemon_details("Staryu")
pokemon.loc[120] = staryu_data
starmie_data = scrap_pokemon_details("Starmie")
pokemon.loc[121] = starmie_data

In [29]:
# Mr mime line 
mrmime_data = scrap_pokemon_details("Mr._Mime")
mrmime_data["ability"] = "Soundproof / Filter"
mrmime_data["hidden_ability"] = "Technician"
mrmime_data["category"] = "Barrier Pokemon"
pokemon.loc[122] = mrmime_data

In [233]:
# Scyther and jynx line 
scyther_data = scrap_pokemon_details("Scyther")
pokemon.loc[123] = scyther_data
jynx_data = scrap_pokemon_details("Jynx")
pokemon.loc[124] = jynx_data

In [234]:
# Electabuzz and Magmar and pinsir
electabuzz_data = scrap_pokemon_details("Electabuzz")
pokemon.loc[125] = electabuzz_data
magmar_data = scrap_pokemon_details("Magmar")
pokemon.loc[126] = magmar_data
pinsir_data = scrap_pokemon_details("Pinsir")
pinsir_data["ability"] = "Hyper Cutter / Mold Breaker"
pinsir_data["hidden_ability"] = "Moxie"
pokemon.loc[127] = pinsir_data

In [235]:
# Tauros and magicarp line
tauros_data = scrap_pokemon_details("Tauros")
tauros_data["ability"] = "Intimidate / Anger Point"
tauros_data["hidden_ability"] = "Sheer Force"
pokemon.loc[128] = tauros_data
magikarp_data = scrap_pokemon_details("Magikarp")
pokemon.loc[129] = magikarp_data
gyarados_data = scrap_pokemon_details("Gyarados")
gyarados_data["ability"] = "Intimidate"
gyarados_data["hidden_ability"] = "Moxie"
pokemon.loc[130] = gyarados_data

In [236]:
# Lapras, ditto
lapras_data = scrap_pokemon_details("Lapras")
pokemon.loc[131] = lapras_data
ditto_data = scrap_pokemon_details("Ditto")
pokemon.loc[132] = ditto_data

In [237]:
# Eevee line
eevee_data = scrap_pokemon_details("Eevee")
pokemon.loc[133] = eevee_data
vaporeon_data = scrap_pokemon_details("Vaporeon")
pokemon.loc[134] = vaporeon_data
jolteon_data = scrap_pokemon_details("Jolteon")
pokemon.loc[135] = jolteon_data
flareon_data = scrap_pokemon_details("Flareon")
pokemon.loc[136] = flareon_data

In [238]:
# porygon and omanyte and kabuto line 
porygon_data = scrap_pokemon_details("Porygon")
pokemon.loc[137] = porygon_data
omanyte_data = scrap_pokemon_details("Omanyte")
pokemon.loc[138] = omanyte_data
omastar_data = scrap_pokemon_details("Omastar")
pokemon.loc[139] = omastar_data
kabuto_data = scrap_pokemon_details("Kabuto")
pokemon.loc[140] = kabuto_data
kabutops_data = scrap_pokemon_details("Kabutops")
pokemon.loc[141] = kabutops_data

In [239]:
# Aerodactyl line and snorlax line
aerodactyl_data = scrap_pokemon_details("Aerodactyl")
aerodactyl_data["ability"] = "Rock Head / Pressure"
aerodactyl_data["hidden_ability"] = "Unnerve"
pokemon.loc[142] = aerodactyl_data
snorlax_data = scrap_pokemon_details("Snorlax")
pokemon.loc[143] = snorlax_data

In [240]:
# Legendary birds
articuno_data = scrap_pokemon_details("Articuno")
articuno_data["ability"] = "Pressure"
articuno_data["hidden_ability"] = "Snow Cloak"
articuno_data["category"] = "Freeze Pokemon" 
pokemon.loc[144] = articuno_data
zapdos_data = scrap_pokemon_details("Zapdos")
zapdos_data["ability"] = "Pressure"
zapdos_data["hidden_ability"] = "Static"
zapdos_data["category"] = "Electric Pokemon" 
pokemon.loc[145] = zapdos_data
moltres_data = scrap_pokemon_details("Moltres")
moltres_data["ability"] = "Pressure"
moltres_data["hidden_ability"] = "Flame Body"
moltres_data["category"] = "Flame Pokemon" 
pokemon.loc[146] = moltres_data

In [248]:
# Dratini line 
dratini_data = scrap_pokemon_details("Dratini")
pokemon.loc[147] = dratini_data
dragonair_data = scrap_pokemon_details("Dragonair")
pokemon.loc[148] = dragonair_data
dragonite_data = scrap_pokemon_details("Dragonite")
pokemon.loc[149] = dragonite_data

In [250]:
# Mew and mewtwo 
mewtwo_data = scrap_pokemon_details("Mewtwo")
mewtwo_data["ability"] = "Pressure"
mewtwo_data["hidden_ability"] = "Unnerve"
pokemon.loc[150] = mewtwo_data
mew_data = scrap_pokemon_details("Mew")
pokemon.loc[151] = mew_data

In [254]:
# Chikorita line 
chikorita = scrap_pokemon_details("Chikorita")
pokemon.loc[152] = chikorita
bayleef = scrap_pokemon_details("Bayleef")
pokemon.loc[153] = bayleef
meganium = scrap_pokemon_details("Meganium")
pokemon.loc[154] = meganium

In [257]:
# Cyndaquil line
cyndaquil = scrap_pokemon_details("Cyndaquil")
pokemon.loc[155] = cyndaquil
quilava = scrap_pokemon_details("Quilava")
pokemon.loc[156] = quilava
typhlosion = scrap_pokemon_details("Typhlosion")
typhlosion["ability"] = "Blaze"
typhlosion["hidden_ability"] = "Flash Fire"
typhlosion["category"] = "Volcano Pokemon"
pokemon.loc[157] = typhlosion

In [259]:
# totodile line 
totodile = scrap_pokemon_details("Totodile")
pokemon.loc[158] = totodile
croconaw = scrap_pokemon_details("Croconaw")
pokemon.loc[159] = croconaw
feraligatr = scrap_pokemon_details("Feraligatr")
pokemon.loc[160] = feraligatr

In [261]:
# sentret line 
sentret = scrap_pokemon_details("Sentret")
pokemon.loc[161] = sentret
furret = scrap_pokemon_details("Furret")
pokemon.loc[162] = furret

In [263]:
# hoothoot line 
hoothoot = scrap_pokemon_details("Hoothoot")
pokemon.loc[163] = hoothoot
noctowl = scrap_pokemon_details("Noctowl")
pokemon.loc[164] = noctowl

In [265]:
# ledyba and spinarak line 
ledyba = scrap_pokemon_details("Ledyba")
pokemon.loc[165] = ledyba
ledian = scrap_pokemon_details("Ledian")
pokemon.loc[166] = ledian
spinarak = scrap_pokemon_details("Spinarak")
pokemon.loc[167] = spinarak
ariados = scrap_pokemon_details("Ariados")
pokemon.loc[168] = ariados

In [267]:
# Crobat and chichou line 
crobat = scrap_pokemon_details("Crobat")
pokemon.loc[169] = crobat
chinchou = scrap_pokemon_details("Chinchou")
pokemon.loc[170] = chinchou
lanturn = scrap_pokemon_details("Lanturn")
pokemon.loc[171] = lanturn

In [269]:
# baby pokemon
pichu = scrap_pokemon_details("Pichu")
pokemon.loc[172] = pichu
cleffa = scrap_pokemon_details("Cleffa")
pokemon.loc[173] = cleffa
igglybuff = scrap_pokemon_details("Igglybuff")
pokemon.loc[174] = igglybuff
togepi = scrap_pokemon_details("Togepi")
pokemon.loc[175] = togepi 
togetic = scrap_pokemon_details("Togetic")
pokemon.loc[176] = togetic

In [271]:
# natu line 
natu = scrap_pokemon_details("Natu")
pokemon.loc[177] = natu
xatu = scrap_pokemon_details("Xatu")
pokemon.loc[178] = xatu

In [275]:
# mareep line
mareep = scrap_pokemon_details("Mareep")
pokemon.loc[179] = mareep
flaaffy = scrap_pokemon_details("Flaaffy")
pokemon.loc[180] = flaaffy
ampharos = scrap_pokemon_details("Ampharos")
ampharos["ability"] = "Static"
ampharos["hidden_ability"] = "Plus"
pokemon.loc[181] = ampharos

In [277]:
# bellossom and maril
bellossom = scrap_pokemon_details("Bellossom")
pokemon.loc[182] = bellossom
marill = scrap_pokemon_details("Marill")
pokemon.loc[183] = marill
azumarill = scrap_pokemon_details("Azumarill")
pokemon.loc[184] = azumarill
sudowoodo = scrap_pokemon_details("Sudowoodo")
pokemon.loc[185] = sudowoodo
politoed = scrap_pokemon_details("Politoed")
pokemon.loc[186] = politoed

In [12]:
# hopip line
hoppip = scrap_pokemon_details("Hoppip")
pokemon.loc[187] = hoppip
skiploom = scrap_pokemon_details("Skiploom")
pokemon.loc[188] = skiploom
jumpluff = scrap_pokemon_details("Jumpluff")
pokemon.loc[189] = jumpluff

In [14]:
# Aipom, sunkern
aipom = scrap_pokemon_details("Aipom")
pokemon.loc[190] = aipom
sunkern = scrap_pokemon_details("Sunkern")
pokemon.loc[191] = sunkern
sunflora = scrap_pokemon_details("Sunflora")
pokemon.loc[192] = sunflora
yanma = scrap_pokemon_details("Yanma")
pokemon.loc[193] = yanma

In [16]:
# wooper
wooper = scrap_pokemon_details("Wooper")
wooper["category"] = "Water Fish Pokemon"
wooper["ability"] = "Damp / Water Absorb"
wooper["hidden_ability"] = "Unaware"
pokemon.loc[194] = wooper
quagsire = scrap_pokemon_details("Quagsire")
pokemon.loc[195] = quagsire

In [18]:
# Eevvelutions
espeon = scrap_pokemon_details("Espeon")
pokemon.loc[196] = espeon
umbreon = scrap_pokemon_details("Umbreon")
pokemon.loc[197] = umbreon
murkrow = scrap_pokemon_details("Murkrow")
pokemon.loc[198] = murkrow

In [63]:
# Slowking
slowking = scrap_pokemon_details("Slowking")
slowking["ability"] = "Oblivious / Own Tempo"
slowking["hidden_ability"] = "Regenerator"
slowking["category"] = "Royal Pokemon"
pokemon.loc[199] = slowking
misdreavus = scrap_pokemon_details("Misdreavus")
pokemon.loc[200] = misdreavus
unown = scrap_pokemon_details("Unown")
pokemon.loc[201] = unown
wobbuffet = scrap_pokemon_details("Wobbuffet")
pokemon.loc[202] = wobbuffet
girafarig = scrap_pokemon_details("Girafarig")
pokemon.loc[203] = girafarig

In [22]:
# Pineco line
pineco = scrap_pokemon_details("Pineco")
pokemon.loc[204] = pineco
forretress = scrap_pokemon_details("Forretress")
pokemon.loc[205] = forretress
pokemon.loc[206] = scrap_pokemon_details("Dunsparce")

In [24]:
# gligar, sleelix
pokemon.loc[207] = scrap_pokemon_details("Gligar")
steelix = scrap_pokemon_details("Steelix")
steelix["ability"] = "Rock Head / Sturdy"
steelix["hidden_ability"] = "Sheer Force"
pokemon.loc[208] = steelix
pokemon.loc[209] = scrap_pokemon_details("Snubbull")
pokemon.loc[210] = scrap_pokemon_details("Granbull")

In [27]:
# Qilfish and scizor
pokemon.loc[211] = scrap_pokemon_details("Qwilfish")
scizor = scrap_pokemon_details("Scizor")
scizor["ability"] = "Swarm / Technician"
scizor["hidden_ability"] = "Light Metal"
pokemon.loc[212] = scizor
pokemon.loc[213] = scrap_pokemon_details("Shuckle")

In [33]:
# Herracross
heracross = scrap_pokemon_details("Heracross")
heracross["ability"] = "Swarm / Guts"
heracross["hidden_ability"] = "Moxie"
pokemon.loc[214] = heracross
pokemon.loc[215] = scrap_pokemon_details("Sneasel")
pokemon.loc[216] = scrap_pokemon_details("Teddiursa")
pokemon.loc[217] = scrap_pokemon_details("Ursaring")
pokemon.loc[218] = scrap_pokemon_details("Slugma")
pokemon.loc[219] = scrap_pokemon_details("Magcargo")
pokemon.loc[220] = scrap_pokemon_details("Swinub")
pokemon.loc[221] = scrap_pokemon_details("Piloswine")

In [35]:
# corsola - houndoom
corsola = scrap_pokemon_details("Corsola")
corsola["ability"] = "Hustle / Natural Cure"
corsola["hidden_ability"] = "Regenerator"
pokemon.loc[222] = corsola
pokemon.loc[223] = scrap_pokemon_details("Remoraid")
pokemon.loc[224] = scrap_pokemon_details("Octillery")
pokemon.loc[225] = scrap_pokemon_details("Delibird")
pokemon.loc[226] = scrap_pokemon_details("Mantine")
pokemon.loc[227] = scrap_pokemon_details("Skarmory")
pokemon.loc[228] = scrap_pokemon_details("Houndour")
houndoom = scrap_pokemon_details("Houndoom")
houndoom["ability"] = "Early Bird / Flash Fire"
houndoom["hidden_ability"] = "Unnerve"
pokemon.loc[229] = houndoom

In [38]:
# kingdra
pokemon.loc[230] = scrap_pokemon_details("Kingdra")
pokemon.loc[231] = scrap_pokemon_details("Phanpy")
pokemon.loc[232] = scrap_pokemon_details("Donphan")
pokemon.loc[233] = scrap_pokemon_details("Porygon2")
pokemon.loc[234] = scrap_pokemon_details("Stantler")
pokemon.loc[235] = scrap_pokemon_details("Smeargle")
pokemon.loc[236] = scrap_pokemon_details("Tyrogue")
pokemon.loc[237] = scrap_pokemon_details("Hitmontop")
pokemon.loc[238] = scrap_pokemon_details("Smoochum")
pokemon.loc[239] = scrap_pokemon_details("Elekid")
pokemon.loc[240] = scrap_pokemon_details("Magby")

In [40]:
# Milktank
pokemon.loc[241] = scrap_pokemon_details("Miltank")
pokemon.loc[242] = scrap_pokemon_details("Blissey")

In [44]:
#Legendary beasts
raikou = scrap_pokemon_details("Raikou")
raikou["ability"] = "Pressure"
raikou["hidden_ability"] = "Inner Focus"
pokemon.loc[243] = raikou
entei = scrap_pokemon_details("Entei")
entei["ability"] = "Pressure"
entei["hidden_ability"] = "Inner Focus"
pokemon.loc[244] = entei
suicune = scrap_pokemon_details("Suicune")
suicune["ability"] = "Pressure"
suicune["hidden_ability"] = "Inner Focus"
pokemon.loc[245] = suicune

In [46]:
# larvitar line 
pokemon.loc[246] = scrap_pokemon_details("Larvitar")
pokemon.loc[247] = scrap_pokemon_details("Pupitar")
tyranitar = scrap_pokemon_details("Tyranitar")
tyranitar["ability"] = "Sand Stream"
tyranitar["hidden_ability"] = "Unnerve"
pokemon.loc[248] = tyranitar
pokemon.loc[249] = scrap_pokemon_details("Lugia")
pokemon.loc[250] = scrap_pokemon_details("Ho-Oh")
pokemon.loc[251] = scrap_pokemon_details("Celebi")

In [49]:
# treeko line 
pokemon.loc[252] = scrap_pokemon_details("Treecko")
pokemon.loc[253] = scrap_pokemon_details("Grovyle")
sceptile = scrap_pokemon_details("Sceptile")
sceptile["ability"] = "Overgrow"
sceptile["hidden_ability"] = "Unburden"
pokemon.loc[254] = sceptile

In [52]:
# torchic line
pokemon.loc[255] = scrap_pokemon_details("Torchic")
pokemon.loc[256] = scrap_pokemon_details("Combusken")
blaziken = scrap_pokemon_details("Blaziken")
blaziken["ability"] = "Blaze"
pokemon.loc[257] = blaziken

In [55]:
# mudkip line
pokemon.loc[258] = scrap_pokemon_details("Mudkip")
pokemon.loc[259] = scrap_pokemon_details("Marshtomp")
swampert = scrap_pokemon_details("Swampert")
swampert["ability"] = "Torrent"
swampert["hidden_ability"] = "Damp"
pokemon.loc[260] = swampert

In [8]:
# poochyena
pokemon.loc[261] = scrap_pokemon_details("Poochyena")
pokemon.loc[262] = scrap_pokemon_details("Mightyena")
pokemon.loc[263] = scrap_pokemon_details("Zigzagoon")
pokemon.loc[264] = scrap_pokemon_details("Linoone")

In [10]:
# Wurmple line and lotad and seedot
pokemon.loc[265] = scrap_pokemon_details("Wurmple")
pokemon.loc[266] = scrap_pokemon_details("Silcoon")
pokemon.loc[267] = scrap_pokemon_details("Beautifly")
pokemon.loc[268] = scrap_pokemon_details("Cascoon")
pokemon.loc[269] = scrap_pokemon_details("Dustox")
pokemon.loc[270] = scrap_pokemon_details("Lotad")
pokemon.loc[271] = scrap_pokemon_details("Lombre")
pokemon.loc[272] = scrap_pokemon_details("Ludicolo")
pokemon.loc[273] = scrap_pokemon_details("Seedot")
pokemon.loc[274] = scrap_pokemon_details("Nuzleaf")

In [7]:
# shiftry Tailow line and wingull
shiftry = scrap_pokemon_details("Shiftry")
shiftry["ability"] = "Chlorophyll / Wind Rider"
shiftry["hidden_ability"] = "Pickpocket"
pokemon.loc[275] = shiftry
pokemon.loc[276] = scrap_pokemon_details("Taillow")
pokemon.loc[277] = scrap_pokemon_details("Swellow")
pokemon.loc[278] = scrap_pokemon_details("Wingull")
pokemon.loc[279] = scrap_pokemon_details("Pelipper")

In [22]:
# ralts line 
pokemon.loc[280] = scrap_pokemon_details("Ralts")
pokemon.loc[281] = scrap_pokemon_details("Kirlia")
gardevoir = scrap_pokemon_details("Gardevoir")
gardevoir["ability"] = "Synchronize / Trace"
gardevoir["hidden_ability"] = "Telepathy"
pokemon.loc[282] = gardevoir

In [24]:
# surskit line and shroomish and slakoth
pokemon.loc[283] = scrap_pokemon_details("Surskit")
pokemon.loc[284] = scrap_pokemon_details("Masquerain")
pokemon.loc[285] = scrap_pokemon_details("Shroomish")
pokemon.loc[286] = scrap_pokemon_details("Breloom")
pokemon.loc[287] = scrap_pokemon_details("Slakoth")
pokemon.loc[288] = scrap_pokemon_details("Vigoroth")
pokemon.loc[289] = scrap_pokemon_details("Slaking")

In [26]:
# nincada line and whismur 
pokemon.loc[290] = scrap_pokemon_details("Nincada")
pokemon.loc[291] = scrap_pokemon_details("Ninjask")
pokemon.loc[292] = scrap_pokemon_details("Shedinja")
pokemon.loc[293] = scrap_pokemon_details("Whismur")
pokemon.loc[294] = scrap_pokemon_details("Loudred")
pokemon.loc[295] = scrap_pokemon_details("Exploud")

In [28]:
# makuhita and skitty
pokemon.loc[296] = scrap_pokemon_details("Makuhita")
pokemon.loc[297] = scrap_pokemon_details("Hariyama")
pokemon.loc[298] = scrap_pokemon_details("Azurill")
pokemon.loc[299] = scrap_pokemon_details("Nosepass")
pokemon.loc[300] = scrap_pokemon_details("Skitty")
pokemon.loc[301] = scrap_pokemon_details("Delcatty")

In [30]:
# sableye
sableye = scrap_pokemon_details("Sableye")
sableye["ability"] = "Keen Eye / Stall"
sableye["hidden_ability"] = "Prankster"
pokemon.loc[302] = sableye
mawile = scrap_pokemon_details("Mawile")
mawile["ability"] = "Hyper Cutter / Intimidate"
mawile["hidden_ability"] = "Sheer Force"
pokemon.loc[303] = mawile

In [32]:
# Aron line'
pokemon.loc[304] = scrap_pokemon_details("Aron")
pokemon.loc[305] = scrap_pokemon_details("Lairon")
aggron = scrap_pokemon_details("Aggron")
aggron["ability"] = "Sturdy / Rock Head"
aggron["hidden_ability"] = "Heavy Metal"
pokemon.loc[306] = aggron

In [34]:
# meditate and electric
pokemon.loc[307] = scrap_pokemon_details("Meditite")
medicham = scrap_pokemon_details("Medicham")
medicham["ability"] = "Pure Power"
medicham["hidden_ability"] = "Telepathy"
pokemon.loc[308] = medicham
pokemon.loc[309] = scrap_pokemon_details("Electrike")
manectric = scrap_pokemon_details("Manectric")
manectric["ability"] = "Static / Lightning Rod"
manectric["hidden_ability"] = "Minus"
pokemon.loc[310] = manectric

In [36]:
# plusle
pokemon.loc[311] = scrap_pokemon_details("Plusle")
pokemon.loc[312] = scrap_pokemon_details("Minun")
pokemon.loc[313] = scrap_pokemon_details("Volbeat")
pokemon.loc[314] = scrap_pokemon_details("Illumise")
pokemon.loc[315] = scrap_pokemon_details("Roselia")
pokemon.loc[316] = scrap_pokemon_details("Gulpin")
pokemon.loc[317] = scrap_pokemon_details("Swalot")

In [38]:
# carvanah line
pokemon.loc[318] = scrap_pokemon_details("Carvanha")
sharpedo = scrap_pokemon_details("Sharpedo")
sharpedo["ability"] = "Rough Skin"
sharpedo["hidden_ability"] = "Speed Boost"
pokemon.loc[319] = sharpedo
pokemon.loc[320] = scrap_pokemon_details("Wailmer")
pokemon.loc[321] = scrap_pokemon_details("Wailord")

In [40]:
# numel
pokemon.loc[322] = scrap_pokemon_details("Numel")
camerupt = scrap_pokemon_details("Camerupt")
camerupt["ability"] = "Magma Armor / Solid Rock"
camerupt["hidden_ability"] = "Anger Point"
pokemon.loc[323] = camerupt
pokemon.loc[324] = scrap_pokemon_details("Torkoal")
pokemon.loc[325] = scrap_pokemon_details("Spoink")
pokemon.loc[326] = scrap_pokemon_details("Grumpig")
pokemon.loc[327] = scrap_pokemon_details("Spinda")

In [43]:
# trapinch to swablu
pokemon.loc[328] = scrap_pokemon_details("Trapinch")
pokemon.loc[329] = scrap_pokemon_details("Vibrava")
pokemon.loc[330] = scrap_pokemon_details("Flygon")
pokemon.loc[331] = scrap_pokemon_details("Cacnea")
pokemon.loc[332] = scrap_pokemon_details("Cacturne")
pokemon.loc[333] = scrap_pokemon_details("Swablu")
altaria = scrap_pokemon_details("Altaria")
altaria["ability"] = "Natural Cure"
altaria["hidden_ability"] = "Cloud Nine"
pokemon.loc[334] = altaria

In [46]:
# zangoose
pokemon.loc[335] = scrap_pokemon_details("Zangoose")
pokemon.loc[336] = scrap_pokemon_details("Seviper")
pokemon.loc[337] = scrap_pokemon_details("Lunatone")
pokemon.loc[338] = scrap_pokemon_details("Solrock")
pokemon.loc[339] = scrap_pokemon_details("Barboach")
pokemon.loc[340] = scrap_pokemon_details("Whiscash")
pokemon.loc[341] = scrap_pokemon_details("Corphish")
pokemon.loc[342] = scrap_pokemon_details("Crawdaunt")
pokemon.loc[343] = scrap_pokemon_details("Baltoy")
pokemon.loc[344] = scrap_pokemon_details("Claydol")

In [48]:
# fossils 3rd gen
pokemon.loc[345] = scrap_pokemon_details("lileep")
pokemon.loc[346] = scrap_pokemon_details("Cradily")
pokemon.loc[347] = scrap_pokemon_details("Anorith")
pokemon.loc[348] = scrap_pokemon_details("Armaldo")
pokemon.loc[349] = scrap_pokemon_details("Feebas")
pokemon.loc[350] = scrap_pokemon_details("Milotic")

In [50]:
# castform
pokemon.loc[351] = scrap_pokemon_details("Castform")

In [52]:
# kecleon
pokemon.loc[352] = scrap_pokemon_details("Kecleon")
pokemon.loc[353] = scrap_pokemon_details("Shuppet")
banette = scrap_pokemon_details("Banette")
banette["ability"] = "Insomnia / Frisk"
banette["hidden_ability"] = "Cursed Body"
pokemon.loc[354] = banette
pokemon.loc[355] = scrap_pokemon_details("Duskull")
pokemon.loc[356] = scrap_pokemon_details("Dusclops")
pokemon.loc[357] = scrap_pokemon_details("Tropius")
pokemon.loc[358] = scrap_pokemon_details("Chimecho")

In [54]:
# absol and wynaut
absol = scrap_pokemon_details("Absol")
absol["ability"] = "Pressure / Super Luck"
absol["hidden_ability"] = "Justified"
pokemon.loc[359] = absol
pokemon.loc[360] = scrap_pokemon_details("Wynaut")
pokemon.loc[361] = scrap_pokemon_details("Snorunt")
glalie = scrap_pokemon_details("Glalie")
glalie["ability"] = "Inner Focus / Ice Body"
glalie["hidden_ability"] = "Moody"
pokemon.loc[362] = glalie

In [57]:
# Spheal line
pokemon.loc[363] = scrap_pokemon_details("Spheal")
pokemon.loc[364] = scrap_pokemon_details("Sealeo")
pokemon.loc[365] = scrap_pokemon_details("Walrein")
pokemon.loc[366] = scrap_pokemon_details("Clamperl")
pokemon.loc[367] = scrap_pokemon_details("Huntail")
pokemon.loc[368] = scrap_pokemon_details("Gorebyss")
pokemon.loc[369] = scrap_pokemon_details("Relicanth")
pokemon.loc[370] = scrap_pokemon_details("Luvdisc")

In [59]:
# bagon line and beldum
pokemon.loc[371] = scrap_pokemon_details("Bagon")
pokemon.loc[372] = scrap_pokemon_details("Shelgon")
salamence = scrap_pokemon_details("Salamence")
salamence["ability"] = "Intimidate"
salamence["hidden_ability"] = "Moxie"
pokemon.loc[373] = salamence
pokemon.loc[374] = scrap_pokemon_details("Beldum")
pokemon.loc[375] = scrap_pokemon_details("Metang")
metagross = scrap_pokemon_details("Metagross")
metagross["ability"] = "Clear Body"
metagross["hidden_ability"] = "Light Metal"
pokemon.loc[376] = metagross

In [63]:
# Regis
pokemon.loc[377] = scrap_pokemon_details("Regirock")
pokemon.loc[378] = scrap_pokemon_details("Regice")
pokemon.loc[379] = scrap_pokemon_details("Registeel")

In [65]:
# latias and latious and other legendaries 
pokemon.loc[380] = scrap_pokemon_details("Latias")
pokemon.loc[381] = scrap_pokemon_details("Latios")
kyogre = scrap_pokemon_details("Kyogre")
kyogre["hidden_ability"] = "None"
pokemon.loc[382] = kyogre
groudon = scrap_pokemon_details("Groudon")
groudon["hidden_ability"] = "None"
pokemon.loc[383] = groudon
rayquaza = scrap_pokemon_details("Rayquaza")
rayquaza["hidden_ability"] = "None"
pokemon.loc[384] = rayquaza
pokemon.loc[385] = scrap_pokemon_details("Jirachi")
pokemon.loc[386] = scrap_pokemon_details("Deoxys")

In [67]:
# Gen 4 starters
pokemon.loc[387] = scrap_pokemon_details("Turtwig")
pokemon.loc[388] = scrap_pokemon_details("Grotle")
pokemon.loc[389] = scrap_pokemon_details("Torterra")
pokemon.loc[390] = scrap_pokemon_details("Chimchar")
pokemon.loc[391] = scrap_pokemon_details("Monferno")
pokemon.loc[392] = scrap_pokemon_details("Infernape")
piplup = scrap_pokemon_details("Piplup")
piplup["ability"] = "Torrent"
piplup["hidden_ability"] = "Competitive"
pokemon.loc[393] = piplup
prinplup = scrap_pokemon_details("Prinplup")
prinplup["ability"] = "Torrent"
prinplup["hidden_ability"] = "Competitive"
pokemon.loc[394] = prinplup
empoleon = scrap_pokemon_details("Empoleon")
empoleon["ability"] = "Torrent"
empoleon["hidden_ability"] = "Competitive"
pokemon.loc[395] = empoleon

In [69]:
# starly and bidoof
pokemon.loc[396] = scrap_pokemon_details("Starly")
pokemon.loc[397] = scrap_pokemon_details("Staravia")
pokemon.loc[398] = scrap_pokemon_details("Staraptor")
pokemon.loc[399] = scrap_pokemon_details("Bidoof")
pokemon.loc[400] = scrap_pokemon_details("Bibarel")
pokemon.loc[401] = scrap_pokemon_details("Kricketot")
pokemon.loc[402] = scrap_pokemon_details("Kricketune")

In [72]:
# Shinx to fossils
pokemon.loc[403] = scrap_pokemon_details("Shinx")
pokemon.loc[404] = scrap_pokemon_details("Luxio")
pokemon.loc[405] = scrap_pokemon_details("Luxray")
pokemon.loc[406] = scrap_pokemon_details("Budew")
pokemon.loc[407] = scrap_pokemon_details("Roserade")
pokemon.loc[408] = scrap_pokemon_details("Cranidos")
pokemon.loc[409] = scrap_pokemon_details("Rampardos")
pokemon.loc[410] = scrap_pokemon_details("Shieldon")
pokemon.loc[411] = scrap_pokemon_details("Bastiodon")

In [74]:
# burmy line
pokemon.loc[412] = scrap_pokemon_details("Burmy")
pokemon.loc[413] = scrap_pokemon_details("Wormadam")
pokemon.loc[414] = scrap_pokemon_details("Mothim")
pokemon.loc[415] = scrap_pokemon_details("Combee")
pokemon.loc[416] = scrap_pokemon_details("Vespiquen")
pokemon.loc[417] = scrap_pokemon_details("Pachirisu")
pokemon.loc[418] = scrap_pokemon_details("Buizel")
pokemon.loc[419] = scrap_pokemon_details("Floatzel")

In [76]:
# cherubi
pokemon.loc[420] = scrap_pokemon_details("Cherubi")
pokemon.loc[421] = scrap_pokemon_details("Cherrim")
pokemon.loc[422] = scrap_pokemon_details("Shellos")
pokemon.loc[423] = scrap_pokemon_details("Gastrodon")
pokemon.loc[424] = scrap_pokemon_details("Ambipom")
pokemon.loc[425] = scrap_pokemon_details("Drifloon")
pokemon.loc[426] = scrap_pokemon_details("Drifblim")

In [78]:
# buneary
pokemon.loc[427] = scrap_pokemon_details("Buneary")
lopunny = scrap_pokemon_details("Lopunny")
lopunny["ability"] = "Cute Charm / Klutz"
lopunny["hidden_ability"] = "Limber"
pokemon.loc[428] = lopunny
pokemon.loc[429] = scrap_pokemon_details("Mismagius")
pokemon.loc[430] = scrap_pokemon_details("Honchkrow")
pokemon.loc[431] = scrap_pokemon_details("Glameow")
pokemon.loc[432] = scrap_pokemon_details("Purugly")
pokemon.loc[433] = scrap_pokemon_details("Chingling")
pokemon.loc[434] = scrap_pokemon_details("Stunky")
pokemon.loc[435] = scrap_pokemon_details("Skuntank")
pokemon.loc[436] = scrap_pokemon_details("Bronzor")
pokemon.loc[437] = scrap_pokemon_details("Bronzong")
pokemon.loc[438] = scrap_pokemon_details("Bonsly")

In [80]:
# MRmime jr
pokemon.loc[439] = scrap_pokemon_details("Mime_Jr.")
pokemon.loc[440] = scrap_pokemon_details("Happiny")
pokemon.loc[441] = scrap_pokemon_details("Chatot")
pokemon.loc[442] = scrap_pokemon_details("Spiritomb")

In [83]:
# Gible line 
pokemon.loc[443] = scrap_pokemon_details("Gible")
pokemon.loc[444] = scrap_pokemon_details("Gabite")
garchomp = scrap_pokemon_details("Garchomp")
garchomp["ability"] = "Sand Veil"
garchomp["hidden_ability"] = "Rough Skin"
pokemon.loc[445] = garchomp
pokemon.loc[446] = scrap_pokemon_details("Munchlax")

In [85]:
# riolu line
pokemon.loc[447] = scrap_pokemon_details("Riolu")
lucario = scrap_pokemon_details("Lucario")
lucario["ability"] = "Steadfast / Inner Focus"
lucario["hidden_ability"] = "Justified"
pokemon.loc[448] = lucario
pokemon.loc[449] = scrap_pokemon_details("Hippopotas")
pokemon.loc[450] = scrap_pokemon_details("Hippowdon")
pokemon.loc[451] = scrap_pokemon_details("Skorupi")
pokemon.loc[452] = scrap_pokemon_details("Drapion")

In [87]:
# croagunk
pokemon.loc[453] = scrap_pokemon_details("Croagunk")
pokemon.loc[454] = scrap_pokemon_details("Toxicroak")
pokemon.loc[455] = scrap_pokemon_details("Carnivine")
pokemon.loc[456] = scrap_pokemon_details("Finneon")
pokemon.loc[457] = scrap_pokemon_details("Lumineon")
pokemon.loc[458] = scrap_pokemon_details("Mantyke")

In [90]:
# Snover line and new evos
pokemon.loc[459] = scrap_pokemon_details("Snover")
abomasnow = scrap_pokemon_details("Abomasnow")
abomasnow["ability"] = "Snow Warning"
abomasnow["hidden_ability"] = "Soundproof"
pokemon.loc[460] = abomasnow
pokemon.loc[461] = scrap_pokemon_details("Weavile")
pokemon.loc[462] = scrap_pokemon_details("Magnezone")
pokemon.loc[463] = scrap_pokemon_details("Lickilicky")
pokemon.loc[464] = scrap_pokemon_details("Rhyperior")
pokemon.loc[465] = scrap_pokemon_details("Tangrowth")
pokemon.loc[466] = scrap_pokemon_details("Electivire")
pokemon.loc[467] = scrap_pokemon_details("Magmortar")
pokemon.loc[468] = scrap_pokemon_details("Togekiss")
pokemon.loc[469] = scrap_pokemon_details("Yanmega")
pokemon.loc[470] = scrap_pokemon_details("Leafeon")
pokemon.loc[471] = scrap_pokemon_details("Glaceon")
pokemon.loc[472] = scrap_pokemon_details("Gliscor")
pokemon.loc[473] = scrap_pokemon_details("Mamoswine")
pokemon.loc[474] = scrap_pokemon_details("Porygon-Z")

In [92]:
# gallad
gallade = scrap_pokemon_details("Gallade")
gallade["ability"] = "Steadfast / Sharpness"
gallade["hidden_ability"] = "Justified"
pokemon.loc[475] = gallade
pokemon.loc[476] = scrap_pokemon_details("Probopass")
pokemon.loc[477] = scrap_pokemon_details("Dusknoir")
pokemon.loc[478] = scrap_pokemon_details("Froslass")
pokemon.loc[479] = scrap_pokemon_details("Rotom")

In [94]:
# legendaries gen 4
pokemon.loc[480] = scrap_pokemon_details("Uxie")
pokemon.loc[481] = scrap_pokemon_details("Mesprit")
pokemon.loc[482] = scrap_pokemon_details("Azelf")
pokemon.loc[483] = scrap_pokemon_details("Dialga")
pokemon.loc[484] = scrap_pokemon_details("Palkia")
pokemon.loc[485] = scrap_pokemon_details("Heatran")
pokemon.loc[486] = scrap_pokemon_details("Regigigas")
giratina = scrap_pokemon_details("Giratina")
giratina["ability"] = "Pressure"
giratina["hidden_ability"] = "Telepathy"
pokemon.loc[487] = giratina
pokemon.loc[488] = scrap_pokemon_details("Cresselia")
pokemon.loc[489] = scrap_pokemon_details("Phione")
pokemon.loc[490] = scrap_pokemon_details("Manaphy")
pokemon.loc[491] = scrap_pokemon_details("Darkrai")
shaymin = scrap_pokemon_details("Shaymin")
shaymin["hidden_ability"] = "None"
pokemon.loc[492] = shaymin
pokemon.loc[493] = scrap_pokemon_details("Arceus")

In [8]:
# gen 5 victi
pokemon.loc[494] = scrap_pokemon_details("Victini")

In [11]:
# gen 5 starters snivy
pokemon.loc[495] = scrap_pokemon_details("Snivy")
pokemon.loc[496] = scrap_pokemon_details("Servine")
pokemon.loc[497] = scrap_pokemon_details("Serperior")

In [12]:
# oshawott and tepig
pokemon.loc[498] = scrap_pokemon_details("Tepig")
pokemon.loc[499] = scrap_pokemon_details("pignite")
pokemon.loc[500] = scrap_pokemon_details("Emboar")
pokemon.loc[501] = scrap_pokemon_details("Oshawott")
pokemon.loc[502] = scrap_pokemon_details("Dewott")
pokemon.loc[503] = scrap_pokemon_details("Samurott")

In [ ]:
# patrat
pokemon.loc[504] = scrap_pokemon_details("Patrat")
pokemon.loc[505] = scrap_pokemon_details("Watchog")
pokemon.loc[506] = scrap_pokemon_details("Lillipup")
pokemon.loc[507] = scrap_pokemon_details("Herdier")
pokemon.loc[508] = scrap_pokemon_details("Stoutland")
pokemon.loc[509] = scrap_pokemon_details("Purrloin")
pokemon.loc[510] = scrap_pokemon_details("Liepard")

In [8]:
#monkeys
pokemon.loc[511] = scrap_pokemon_details("Pansage")
pokemon.loc[512] = scrap_pokemon_details("Simisage")
pokemon.loc[513] = scrap_pokemon_details("Pansear")
pokemon.loc[514] = scrap_pokemon_details("Simisear")
pokemon.loc[515] = scrap_pokemon_details("Panpour")
pokemon.loc[516] = scrap_pokemon_details("Simipour")

In [9]:
# muna and pidove
pokemon.loc[517] = scrap_pokemon_details("Munna")
pokemon.loc[518] = scrap_pokemon_details("Musharna")
pokemon.loc[519] = scrap_pokemon_details("Pidove")
pokemon.loc[520] = scrap_pokemon_details("Tranquill")
pokemon.loc[521] = scrap_pokemon_details("Unfezant")
pokemon.loc[522] = scrap_pokemon_details("Blitzle")
pokemon.loc[523] = scrap_pokemon_details("Zebstrika")

In [10]:
#roggenrola
pokemon.loc[524] = scrap_pokemon_details("Roggenrola")
pokemon.loc[525] = scrap_pokemon_details("Boldore")
pokemon.loc[526] = scrap_pokemon_details("Gigalith")
pokemon.loc[527] = scrap_pokemon_details("Woobat")
pokemon.loc[528] = scrap_pokemon_details("Swoobat")
pokemon.loc[529] = scrap_pokemon_details("Drilbur")
pokemon.loc[530] = scrap_pokemon_details("Excadrill")
audino = scrap_pokemon_details("Audino")
audino["ability"] = "Healer / Regenerator"
audino["hidden_ability"] = "Klutz"
pokemon.loc[531] = audino

In [11]:
# timburr and tympole
pokemon.loc[532] = scrap_pokemon_details("Timburr")
pokemon.loc[533] = scrap_pokemon_details("Gurdurr")
pokemon.loc[534] = scrap_pokemon_details("Conkeldurr")
pokemon.loc[535] = scrap_pokemon_details("Tympole")
pokemon.loc[536] = scrap_pokemon_details("Palpitoad")
pokemon.loc[537] = scrap_pokemon_details("Seismitoad")
pokemon.loc[538] = scrap_pokemon_details("Throh")
pokemon.loc[539] = scrap_pokemon_details("Sawk")

In [12]:
# swaddle
pokemon.loc[540] = scrap_pokemon_details("Sewaddle")
pokemon.loc[541] = scrap_pokemon_details("Swadloon")
pokemon.loc[542] = scrap_pokemon_details("Leavanny")
venipede = scrap_pokemon_details("Venipede")
venipede["ability"] = "Poison Point / Swarm"
venipede["hidden_ability"] = "Speed Boost"
pokemon.loc[543] = venipede
whirlipede = scrap_pokemon_details("Whirlipede")
whirlipede["ability"] = "Poison Point / Swarm"
whirlipede["hidden_ability"] = "Speed Boost"
pokemon.loc[544] = whirlipede
scolipede = scrap_pokemon_details("Scolipede")
scolipede["ability"] = "Poison Point / Swarm"
scolipede["hidden_ability"] = "Speed Boost"
pokemon.loc[545] = scolipede

In [13]:
# cottonee
pokemon.loc[546] = scrap_pokemon_details("Cottonee")
pokemon.loc[547] = scrap_pokemon_details("Whimsicott")
pokemon.loc[548] = scrap_pokemon_details("Petilil")

In [14]:
# lilligant
lilligant = scrap_pokemon_details("Lilligant")
lilligant["ability"] = "Chlorophyll / Own Tempo"
lilligant["hidden_ability"] = "Leaf Guard"
lilligant["category"] = "Flowering Pokemon"
pokemon.loc[549] = lilligant
basculin = scrap_pokemon_details("Basculin")
basculin["ability"] = "Reckless / Adaptability"
basculin["hidden_ability"] = "Mold Breaker"
basculin["category"] = "Hostile Pokemon"
pokemon.loc[550] = basculin

In [15]:
# sandile
pokemon.loc[551] = scrap_pokemon_details("Sandile")
pokemon.loc[552] = scrap_pokemon_details("Krokorok")
pokemon.loc[553] = scrap_pokemon_details("Krookodile")
pokemon.loc[554] = scrap_pokemon_details("Darumaka")
darmanitan = scrap_pokemon_details("Darmanitan")
darmanitan["ability"] = "Sheer Force"
darmanitan["category"] = "Blazing Pokemon"
pokemon.loc[555] = darmanitan

In [16]:
# maractus 
pokemon.loc[556] = scrap_pokemon_details("Maractus")
pokemon.loc[557] = scrap_pokemon_details("Dwebble")
pokemon.loc[558] = scrap_pokemon_details("Crustle")
pokemon.loc[559] = scrap_pokemon_details("Scraggy")
pokemon.loc[560] = scrap_pokemon_details("Scrafty")
pokemon.loc[561] = scrap_pokemon_details("Sigilyph")

In [17]:
# yamask
pokemon.loc[562] = scrap_pokemon_details("Yamask")
pokemon.loc[563] = scrap_pokemon_details("Cofagrigus")
pokemon.loc[564] = scrap_pokemon_details("Tirtouga")
pokemon.loc[565] = scrap_pokemon_details("Carracosta")
pokemon.loc[566] = scrap_pokemon_details("Archen")
pokemon.loc[567] = scrap_pokemon_details("Archeops")
pokemon.loc[568] = scrap_pokemon_details("Trubbish")
pokemon.loc[569] = scrap_pokemon_details("Garbodor")

In [18]:
# zorua
pokemon.loc[570] = scrap_pokemon_details("Zorua")
pokemon.loc[570, "category"] = "Tricky Fox Pokemon"
pokemon.loc[571] = scrap_pokemon_details("Zoroark")
pokemon.loc[571, "category"] = "Illusion Fox Pokemon"

In [19]:
# minccinp 
pokemon.loc[572] = scrap_pokemon_details("Minccino")
pokemon.loc[573] = scrap_pokemon_details("Cinccino")
pokemon.loc[574] = scrap_pokemon_details("Gothita")
pokemon.loc[575] = scrap_pokemon_details("Gothorita")
pokemon.loc[576] = scrap_pokemon_details("Gothitelle")
pokemon.loc[577] = scrap_pokemon_details("Solosis")
pokemon.loc[578] = scrap_pokemon_details("Duosion")
pokemon.loc[579] = scrap_pokemon_details("Reuniclus")

In [20]:
#ducklett
pokemon.loc[580] = scrap_pokemon_details("Ducklett")
pokemon.loc[581] = scrap_pokemon_details("Swanna")
pokemon.loc[582] = scrap_pokemon_details("Vanillite")
pokemon.loc[583] = scrap_pokemon_details("Vanillish")
pokemon.loc[584] = scrap_pokemon_details("Vanilluxe")
pokemon.loc[585] = scrap_pokemon_details("Deerling")
pokemon.loc[586] = scrap_pokemon_details("Sawsbuck")

In [21]:
# emolga
pokemon.loc[587] = scrap_pokemon_details("Emolga")
pokemon.loc[588] = scrap_pokemon_details("Karrablast")
pokemon.loc[589] = scrap_pokemon_details("Escavalier")
pokemon.loc[590] = scrap_pokemon_details("Foongus")
pokemon.loc[591] = scrap_pokemon_details("Amoonguss")
pokemon.loc[592] = scrap_pokemon_details("Frillish")
pokemon.loc[593] = scrap_pokemon_details("Jellicent")
pokemon.loc[594] = scrap_pokemon_details("Alomomola")
pokemon.loc[595] = scrap_pokemon_details("Joltik")
pokemon.loc[596] = scrap_pokemon_details("Galvantula")
pokemon.loc[597] = scrap_pokemon_details("Ferroseed")
pokemon.loc[598] = scrap_pokemon_details("Ferrothorn")
pokemon.loc[599] = scrap_pokemon_details("Klink")
pokemon.loc[600] = scrap_pokemon_details("Klang")
pokemon.loc[601] = scrap_pokemon_details("Klinklang")

In [24]:
# tynamo
pokemon.loc[602] = scrap_pokemon_details("Tynamo")
pokemon.loc[603] = scrap_pokemon_details("Eelektrik")
pokemon.loc[604] = scrap_pokemon_details("Eelektross")
pokemon.loc[605] = scrap_pokemon_details("Elgyem")
pokemon.loc[606] = scrap_pokemon_details("Beheeyem")
pokemon.loc[607] = scrap_pokemon_details("Litwick")
pokemon.loc[607, "ability"] = "Flash Fire / Flame Body"
pokemon.loc[607, "hidden_ability"] = "Infiltrator"
pokemon.loc[608] = scrap_pokemon_details("Lampent")
pokemon.loc[608, "ability"] = "Flash Fire / Flame Body"
pokemon.loc[608, "hidden_ability"] = "Infiltrator"
pokemon.loc[609] = scrap_pokemon_details("Chandelure")
pokemon.loc[609, "ability"] = "Flash Fire / Flame Body"
pokemon.loc[609, "hidden_ability"] = "Infiltrator"

In [25]:
# axew
pokemon.loc[610] = scrap_pokemon_details("Axew")
pokemon.loc[611] = scrap_pokemon_details("Fraxure")
pokemon.loc[612] = scrap_pokemon_details("Haxorus")
pokemon.loc[613] = scrap_pokemon_details("Cubchoo")
pokemon.loc[614] = scrap_pokemon_details("Beartic")
pokemon.loc[615] = scrap_pokemon_details("Cryogonal")
pokemon.loc[616] = scrap_pokemon_details("Shelmet")
pokemon.loc[617] = scrap_pokemon_details("Accelgor")

In [26]:
# stunkfish 
pokemon.loc[618] = scrap_pokemon_details("Stunfisk")
pokemon.loc[618, "ability"] = "Static / Limber"
pokemon.loc[618, "hidden_ability"] = "Sand Veil"

In [27]:
# meinfoo
pokemon.loc[619] = scrap_pokemon_details("Mienfoo")
pokemon.loc[620] = scrap_pokemon_details("Mienshao")
pokemon.loc[621] = scrap_pokemon_details("Druddigon")
pokemon.loc[622] = scrap_pokemon_details("Golett")
pokemon.loc[623] = scrap_pokemon_details("Golurk")
pokemon.loc[624] = scrap_pokemon_details("Pawniard")
pokemon.loc[625] = scrap_pokemon_details("Bisharp")
pokemon.loc[626] = scrap_pokemon_details("Bouffalant")
pokemon.loc[627] = scrap_pokemon_details("Rufflet")
pokemon.loc[628] = scrap_pokemon_details("Braviary")
pokemon.loc[628, "ability"] = "Keen Eye / Sheer Force"
pokemon.loc[628, "hidden_ability"] = "Defiant"
pokemon.loc[628, "category"] = "Valiant Pokemon"

In [28]:
# vullaby
pokemon.loc[629] = scrap_pokemon_details("Vullaby")
pokemon.loc[630] = scrap_pokemon_details("Mandibuzz")
pokemon.loc[631] = scrap_pokemon_details("Heatmor")
pokemon.loc[632] = scrap_pokemon_details("Durant")
pokemon.loc[633] = scrap_pokemon_details("Deino")
pokemon.loc[634] = scrap_pokemon_details("Zweilous")
pokemon.loc[635] = scrap_pokemon_details("Hydreigon")
pokemon.loc[636] = scrap_pokemon_details("Larvesta")
pokemon.loc[637] = scrap_pokemon_details("Volcarona")

In [29]:
# legendaries
pokemon.loc[638] = scrap_pokemon_details("Cobalion")
pokemon.loc[639] = scrap_pokemon_details("Terrakion")
pokemon.loc[640] = scrap_pokemon_details("Virizion")
pokemon.loc[641] = scrap_pokemon_details("Tornadus")
pokemon.loc[641, "ability"] = "Prankster"
pokemon.loc[641, "hidden_ability"] = "Defiant"
pokemon.loc[642] = scrap_pokemon_details("Thundurus")
pokemon.loc[642, "ability"] = "Prankster"
pokemon.loc[642, "hidden_ability"] = "Defiant"
pokemon.loc[643] = scrap_pokemon_details("Reshiram")
pokemon.loc[644] = scrap_pokemon_details("Zekrom")
pokemon.loc[645] = scrap_pokemon_details("Landorus")
pokemon.loc[645, "ability"] = "Prankster"
pokemon.loc[645, "hidden_ability"] = "Defiant"
pokemon.loc[646] = scrap_pokemon_details("Kyurem")
pokemon.loc[646, "ability"] = "Pressure"
pokemon.loc[646, "hidden_ability"] = "None"
pokemon.loc[647] = scrap_pokemon_details("Keldeo")
pokemon.loc[648] = scrap_pokemon_details("Meloetta")
pokemon.loc[649] = scrap_pokemon_details("Genesect")

In [30]:
# gen 6 starters 
pokemon.loc[650] = scrap_pokemon_details("Chespin")
pokemon.loc[651] = scrap_pokemon_details("Quilladin")
pokemon.loc[652] = scrap_pokemon_details("Chesnaught")
pokemon.loc[653] = scrap_pokemon_details("Fennekin")
pokemon.loc[654] = scrap_pokemon_details("Braixen")
pokemon.loc[655] = scrap_pokemon_details("Delphox")
pokemon.loc[656] = scrap_pokemon_details("Froakie")
pokemon.loc[657] = scrap_pokemon_details("Frogadier")
pokemon.loc[658] = scrap_pokemon_details("Greninja")

In [31]:
pokemon.loc[658, "egg_group"] = "Water 1"
# bunnelby
pokemon.loc[659] = scrap_pokemon_details("Bunnelby")
pokemon.loc[660] = scrap_pokemon_details("Diggersby")
pokemon.loc[661] = scrap_pokemon_details("Fletchling")
pokemon.loc[662] = scrap_pokemon_details("Fletchinder")
pokemon.loc[663] = scrap_pokemon_details("Talonflame")

In [32]:
# scatterbug
pokemon.loc[664] = scrap_pokemon_details("Scatterbug")
pokemon.loc[665] = scrap_pokemon_details("Spewpa")
pokemon.loc[666] = scrap_pokemon_details("Vivillon")
pokemon.loc[667] = scrap_pokemon_details("Litleo")
pokemon.loc[668] = scrap_pokemon_details("Pyroar")
pokemon.loc[669] = scrap_pokemon_details("Flabébé")
pokemon.loc[670] = scrap_pokemon_details("Floette")
pokemon.loc[671] = scrap_pokemon_details("Florges")

In [33]:
# skiddo
pokemon.loc[672] = scrap_pokemon_details("Skiddo")
pokemon.loc[673] = scrap_pokemon_details("Gogoat")
pokemon.loc[674] = scrap_pokemon_details("Pancham")
pokemon.loc[675] = scrap_pokemon_details("Pangoro")
pokemon.loc[676] = scrap_pokemon_details("Furfrou")
pokemon.loc[677] = scrap_pokemon_details("Espurr")
pokemon.loc[678] = scrap_pokemon_details("Meowstic")
pokemon.loc[678, "ability"] = "Keen Eye / Infiltrator"
pokemon.loc[679] = scrap_pokemon_details("Honedge")
pokemon.loc[680] = scrap_pokemon_details("Doublade")
pokemon.loc[681] = scrap_pokemon_details("Aegislash")

In [ ]:
pokemon.loc[678, "hidden_ability"] = "Prankster / Competitive" 
# spritzee
pokemon.loc[682] = scrap_pokemon_details("Spritzee")
pokemon.loc[683] = scrap_pokemon_details("Aromatisse")
pokemon.loc[684] = scrap_pokemon_details("Swirlix")
pokemon.loc[685] = scrap_pokemon_details("Slurpuff")
pokemon.loc[686] = scrap_pokemon_details("Inkay")
pokemon.loc[687] = scrap_pokemon_details("Malamar")
pokemon.loc[688] = scrap_pokemon_details("Binacle")
pokemon.loc[689] = scrap_pokemon_details("Barbaracle")
pokemon.loc[690] = scrap_pokemon_details("Skrelp")
pokemon.loc[691] = scrap_pokemon_details("Dragalge")
pokemon.loc[692] = scrap_pokemon_details("Clauncher")
pokemon.loc[693] = scrap_pokemon_details("Clawitzer")
pokemon.loc[694] = scrap_pokemon_details("Helioptile")
pokemon.loc[695] = scrap_pokemon_details("Heliolisk")
pokemon.loc[696] = scrap_pokemon_details("Tyrunt")
pokemon.loc[697] = scrap_pokemon_details("Tyrantrum")
pokemon.loc[698] = scrap_pokemon_details("Amaura")
pokemon.loc[699] = scrap_pokemon_details("Aurorus")
pokemon.loc[700] = scrap_pokemon_details("Sylveon")

In [8]:
# hawlucha
pokemon.loc[701] = scrap_pokemon_details("Hawlucha")
pokemon.loc[702] = scrap_pokemon_details("Dedenne")
pokemon.loc[703] = scrap_pokemon_details("Carbink")
pokemon.loc[704] = scrap_pokemon_details("Goomy")
pokemon.loc[705] = scrap_pokemon_details("Sliggoo")
pokemon.loc[705, "ability"] = "Sap Sipper / Hydration"
pokemon.loc[705, "hidden_ability"] = "Gooey"
pokemon.loc[705, "category"] = "Soft Tissue Pokemon"
pokemon.loc[706] = scrap_pokemon_details("Goodra")
pokemon.loc[706, "ability"] = "Sap Sipper / Hydration"
pokemon.loc[706, "hidden_ability"] = "Gooey"
pokemon.loc[706, "category"] = "Dragon Pokemon"

In [9]:
#klefki
pokemon.loc[707] = scrap_pokemon_details("Klefki")
pokemon.loc[708] = scrap_pokemon_details("Phantump")
pokemon.loc[709] = scrap_pokemon_details("Trevenant")
pokemon.loc[710] = scrap_pokemon_details("Pumpkaboo")
pokemon.loc[711] = scrap_pokemon_details("Gourgeist")
pokemon.loc[712] = scrap_pokemon_details("Bergmite")
pokemon.loc[713] = scrap_pokemon_details("Avalugg")
pokemon.loc[713, "ability"] = "Own Tempo / Ice Body"
pokemon.loc[713, "hidden_ability"] = "Sturdy"

In [10]:
# Noibat
pokemon.loc[714] = scrap_pokemon_details("Noibat")
pokemon.loc[715] = scrap_pokemon_details("Noivern")
pokemon.loc[716] = scrap_pokemon_details("Xerneas")
pokemon.loc[717] = scrap_pokemon_details("Yveltal")
pokemon.loc[718] = scrap_pokemon_details("Zygarde")
pokemon.loc[718, "ability"] = "Aura Break"
pokemon.loc[719] = scrap_pokemon_details("Diancie")
pokemon.loc[720] = scrap_pokemon_details("Hoopa")
pokemon.loc[720, "category"] = "Mischief Pokemon"
pokemon.loc[721] = scrap_pokemon_details("Volcanion")

In [11]:
# gen 7 starters 
pokemon.loc[722] = scrap_pokemon_details("Rowlet")
pokemon.loc[723] = scrap_pokemon_details("Dartrix")
pokemon.loc[724] = scrap_pokemon_details("Decidueye")
pokemon.loc[724, "ability"] = "Overgrow"
pokemon.loc[724, "hidden_ability"] = "Long Reach"
pokemon.loc[725] = scrap_pokemon_details("Litten")
pokemon.loc[726] = scrap_pokemon_details("Torracat")
pokemon.loc[727] = scrap_pokemon_details("Incineroar")
pokemon.loc[728] = scrap_pokemon_details("Popplio")
pokemon.loc[729] = scrap_pokemon_details("Brionne")

In [12]:
# pikipek line 
pokemon.loc[730] = scrap_pokemon_details("Primarina")
pokemon.loc[731] = scrap_pokemon_details("Pikipek")
pokemon.loc[732] = scrap_pokemon_details("Trumbeak")
pokemon.loc[733] = scrap_pokemon_details("Toucannon")
pokemon.loc[734] = scrap_pokemon_details("Yungoos")
pokemon.loc[735] = scrap_pokemon_details("Gumshoos")


In [13]:
# grubbin line
pokemon.loc[736] = scrap_pokemon_details("Grubbin")
pokemon.loc[737] = scrap_pokemon_details("Charjabug")
pokemon.loc[738] = scrap_pokemon_details("Vikavolt")
pokemon.loc[739] = scrap_pokemon_details("Crabrawler")
pokemon.loc[740] = scrap_pokemon_details("Crabominable")
pokemon.loc[741] = scrap_pokemon_details("Oricorio")
pokemon.loc[742] = scrap_pokemon_details("Cutiefly")
pokemon.loc[743] = scrap_pokemon_details("Ribombee")

In [14]:
# rockruff
pokemon.loc[744] = scrap_pokemon_details("Rockruff")
pokemon.loc[744, "ability"] = "Keen Eye / Vital Spirit"
pokemon.loc[744, "hidden_ability"] = "Steadfast"
pokemon.loc[745] = scrap_pokemon_details("Lycanroc")
pokemon.loc[745, "ability"] = "Keen Eye / Sand Rush"
pokemon.loc[745, "hidden_ability"] = "Steadfast"
pokemon.loc[746] = scrap_pokemon_details("Wishiwashi")
pokemon.loc[747] = scrap_pokemon_details("Mareanie")
pokemon.loc[748] = scrap_pokemon_details("Toxapex")

In [15]:
# Mudbray
pokemon.loc[749] = scrap_pokemon_details("Mudbray")
pokemon.loc[750] = scrap_pokemon_details("Mudsdale")
pokemon.loc[751] = scrap_pokemon_details("Dewpider")
pokemon.loc[752] = scrap_pokemon_details("Araquanid")
pokemon.loc[753] = scrap_pokemon_details("Fomantis")
pokemon.loc[754] = scrap_pokemon_details("Lurantis")
pokemon.loc[755] = scrap_pokemon_details("Morelull")
pokemon.loc[756] = scrap_pokemon_details("Shiinotic")
pokemon.loc[757] = scrap_pokemon_details("Salandit")
pokemon.loc[758] = scrap_pokemon_details("Salazzle")

In [16]:
# stufful
pokemon.loc[759] = scrap_pokemon_details("Stufful")
pokemon.loc[760] = scrap_pokemon_details("Bewear")
pokemon.loc[761] = scrap_pokemon_details("Bounsweet")
pokemon.loc[762] = scrap_pokemon_details("Steenee")
pokemon.loc[763] = scrap_pokemon_details("Tsareena")

In [17]:
# comfey 
pokemon.loc[764] = scrap_pokemon_details("Comfey")
pokemon.loc[765] = scrap_pokemon_details("Oranguru")
pokemon.loc[766] = scrap_pokemon_details("Passimian")
pokemon.loc[767] = scrap_pokemon_details("Wimpod")
pokemon.loc[768] = scrap_pokemon_details("Golisopod")
pokemon.loc[769] = scrap_pokemon_details("Sandygast")
pokemon.loc[770] = scrap_pokemon_details("Palossand")

In [18]:
# pyukumuku
pokemon.loc[771] = scrap_pokemon_details("Pyukumuku")
pokemon.loc[772] = scrap_pokemon_details("Type:_Null")
pokemon.loc[773] = scrap_pokemon_details("Silvally")
pokemon.loc[774] = scrap_pokemon_details("Minior")
pokemon.loc[775] = scrap_pokemon_details("Komala")
pokemon.loc[776] = scrap_pokemon_details("Turtonator")
pokemon.loc[777] = scrap_pokemon_details("Togedemaru")
pokemon.loc[778] = scrap_pokemon_details("Mimikyu")

In [19]:
# bruxish 
pokemon.loc[779] = scrap_pokemon_details("Bruxish")
pokemon.loc[780] = scrap_pokemon_details("Drampa")
pokemon.loc[781] = scrap_pokemon_details("Dhelmise")
pokemon.loc[782] = scrap_pokemon_details("Jangmo-o")
pokemon.loc[783] = scrap_pokemon_details("Hakamo-o")
pokemon.loc[784] = scrap_pokemon_details("Kommo-o")

In [20]:
# legendaries
pokemon.loc[785] = scrap_pokemon_details("Tapu_Koko")
pokemon.loc[786] = scrap_pokemon_details("Tapu_Lele")
pokemon.loc[787] = scrap_pokemon_details("Tapu_Bulu")
pokemon.loc[788] = scrap_pokemon_details("Tapu_Fini")
pokemon.loc[789] = scrap_pokemon_details("Cosmog")
pokemon.loc[790] = scrap_pokemon_details("Cosmoem")
pokemon.loc[791] = scrap_pokemon_details("Solgaleo")
pokemon.loc[792] = scrap_pokemon_details("Lunala")

In [21]:
# ultra beasts
pokemon.loc[793] = scrap_pokemon_details("Nihilego")
pokemon.loc[794] = scrap_pokemon_details("Buzzwole")
pokemon.loc[795] = scrap_pokemon_details("Pheromosa")
pokemon.loc[796] = scrap_pokemon_details("Xurkitree")
pokemon.loc[797] = scrap_pokemon_details("Celesteela")
pokemon.loc[798] = scrap_pokemon_details("Kartana")
pokemon.loc[799] = scrap_pokemon_details("Guzzlord")
pokemon.loc[800] = scrap_pokemon_details("Necrozma")
pokemon.loc[801] = scrap_pokemon_details("Magearna")
pokemon.loc[802] = scrap_pokemon_details("Marshadow")
pokemon.loc[803] = scrap_pokemon_details("Poipole")
pokemon.loc[804] = scrap_pokemon_details("Naganadel")
pokemon.loc[805] = scrap_pokemon_details("Stakataka")
pokemon.loc[806] = scrap_pokemon_details("Blacephalon")
pokemon.loc[807] = scrap_pokemon_details("Zeraora")
pokemon.loc[808] = scrap_pokemon_details("Meltan")
pokemon.loc[809] = scrap_pokemon_details("Melmetal")

In [22]:
# gen 8 starters 
pokemon.loc[810] = scrap_pokemon_details("Grookey")
pokemon.loc[811] = scrap_pokemon_details("Thwackey")
pokemon.loc[812] = scrap_pokemon_details("Rillaboom")
pokemon.loc[813] = scrap_pokemon_details("Scorbunny")
pokemon.loc[814] = scrap_pokemon_details("Raboot")
pokemon.loc[815] = scrap_pokemon_details("Cinderace")
pokemon.loc[816] = scrap_pokemon_details("Sobble")
pokemon.loc[817] = scrap_pokemon_details("Drizzile")
pokemon.loc[818] = scrap_pokemon_details("Inteleon")

In [23]:
# skwovet
pokemon.loc[819] = scrap_pokemon_details("Skwovet")
pokemon.loc[820] = scrap_pokemon_details("Greedent")
pokemon.loc[821] = scrap_pokemon_details("Rookidee")
pokemon.loc[822] = scrap_pokemon_details("Corvisquire")
pokemon.loc[823] = scrap_pokemon_details("Corviknight")
pokemon.loc[824] = scrap_pokemon_details("Blipbug")
pokemon.loc[825] = scrap_pokemon_details("Dottler")
pokemon.loc[826] = scrap_pokemon_details("Orbeetle")
pokemon.loc[827] = scrap_pokemon_details("Nickit")
pokemon.loc[828] = scrap_pokemon_details("Thievul")

In [24]:
#gossifleur
pokemon.loc[829] = scrap_pokemon_details("Gossifleur")
pokemon.loc[830] = scrap_pokemon_details("Eldegoss")
pokemon.loc[831] = scrap_pokemon_details("Wooloo")
pokemon.loc[832] = scrap_pokemon_details("Dubwool")
pokemon.loc[833] = scrap_pokemon_details("Chewtle")
pokemon.loc[834] = scrap_pokemon_details("Drednaw")
pokemon.loc[835] = scrap_pokemon_details("Yamper")
pokemon.loc[836] = scrap_pokemon_details("Boltund")
pokemon.loc[837] = scrap_pokemon_details("Rolycoly")
pokemon.loc[838] = scrap_pokemon_details("Carkol")
pokemon.loc[839] = scrap_pokemon_details("Coalossal")

In [25]:
# applin'
pokemon.loc[840] = scrap_pokemon_details("Applin")
pokemon.loc[841] = scrap_pokemon_details("Flapple")
pokemon.loc[842] = scrap_pokemon_details("Appletun")
pokemon.loc[843] = scrap_pokemon_details("Silicobra")
pokemon.loc[844] = scrap_pokemon_details("Sandaconda")
pokemon.loc[845] = scrap_pokemon_details("Cramorant")
pokemon.loc[846] = scrap_pokemon_details("Arrokuda")
pokemon.loc[847] = scrap_pokemon_details("Barraskewda")
pokemon.loc[848] = scrap_pokemon_details("Toxel")
pokemon.loc[849] = scrap_pokemon_details("Toxtricity")
pokemon.loc[849, "ability"] = "Punk Rock / Plus"
pokemon.loc[849, "hidden_ability"] = "Technician"

In [26]:
#sizzlipede
pokemon.loc[850] = scrap_pokemon_details("Sizzlipede")
pokemon.loc[851] = scrap_pokemon_details("Centiskorch")
pokemon.loc[852] = scrap_pokemon_details("Clobbopus")
pokemon.loc[853] = scrap_pokemon_details("Grapploct")
pokemon.loc[854] = scrap_pokemon_details("Sinistea")
pokemon.loc[855] = scrap_pokemon_details("Polteageist")
pokemon.loc[856] = scrap_pokemon_details("Hatenna")
pokemon.loc[857] = scrap_pokemon_details("Hattrem")
pokemon.loc[858] = scrap_pokemon_details("Hatterene")

In [27]:
# impidimp
pokemon.loc[859] = scrap_pokemon_details("Impidimp")
pokemon.loc[860] = scrap_pokemon_details("Morgrem")
pokemon.loc[861] = scrap_pokemon_details("Grimmsnarl")
pokemon.loc[862] = scrap_pokemon_details("Obstagoon")
pokemon.loc[863] = scrap_pokemon_details("Perrserker")
pokemon.loc[864] = scrap_pokemon_details("Cursola")
pokemon.loc[865] = scrap_pokemon_details("Sirfetch'd")
pokemon.loc[866] = scrap_pokemon_details("Mr._Rime")
pokemon.loc[867] = scrap_pokemon_details("Runerigus")
pokemon.loc[868] = scrap_pokemon_details("Milcery")
pokemon.loc[869] = scrap_pokemon_details("Alcremie")
pokemon.loc[870] = scrap_pokemon_details("Falinks")
pokemon.loc[871] = scrap_pokemon_details("Pincurchin")

In [28]:
# snom
pokemon.loc[872] = scrap_pokemon_details("Snom")
pokemon.loc[873] = scrap_pokemon_details("Frosmoth")
pokemon.loc[874] = scrap_pokemon_details("Stonjourner")
pokemon.loc[875] = scrap_pokemon_details("Eiscue")
pokemon.loc[876] = scrap_pokemon_details("Indeedee")
pokemon.loc[876, "ability"] = "Own Tempo / Synchronize"
pokemon.loc[876, "hidden_ability"] = "Psychic Surge"
pokemon.loc[877] = scrap_pokemon_details("Morpeko")
pokemon.loc[878] = scrap_pokemon_details("Cufant")
pokemon.loc[879] = scrap_pokemon_details("Copperajah")

In [29]:
# fossils
pokemon.loc[880] = scrap_pokemon_details("Dracozolt")
pokemon.loc[881] = scrap_pokemon_details("Arctozolt")
pokemon.loc[882] = scrap_pokemon_details("Dracovish")
pokemon.loc[883] = scrap_pokemon_details("Arctovish")
pokemon.loc[884] = scrap_pokemon_details("Duraludon")
pokemon.loc[885] = scrap_pokemon_details("Dreepy")
pokemon.loc[886] = scrap_pokemon_details("Drakloak")
pokemon.loc[887] = scrap_pokemon_details("Dragapult")


In [30]:
#legendaries
pokemon.loc[888] = scrap_pokemon_details("Zacian")
pokemon.loc[889] = scrap_pokemon_details("Zamazenta")
pokemon.loc[890] = scrap_pokemon_details("Eternatus")
pokemon.loc[891] = scrap_pokemon_details("Kubfu")
pokemon.loc[892] = scrap_pokemon_details("Urshifu")
pokemon.loc[893] = scrap_pokemon_details("Zarude")
pokemon.loc[894] = scrap_pokemon_details("Regieleki")
pokemon.loc[895] = scrap_pokemon_details("Regidrago")
pokemon.loc[896] = scrap_pokemon_details("Glastrier")
pokemon.loc[897] = scrap_pokemon_details("Spectrier")
pokemon.loc[898] = scrap_pokemon_details("Calyrex")

In [11]:
# hisuin 
pokemon.loc[899] = scrap_pokemon_details("Wyrdeer")
pokemon.loc[900] = scrap_pokemon_details("Kleavor")
pokemon.loc[901] = scrap_pokemon_details("Ursaluna")
pokemon.loc[901, "ability"] = "Guts / Bulletproof"
pokemon.loc[901, "hidden_ability"] = "Unnerve"
pokemon.loc[902] = scrap_pokemon_details("Basculegion")
pokemon.loc[903] = scrap_pokemon_details("Sneasler")
pokemon.loc[904] = scrap_pokemon_details("Overqwil")
pokemon.loc[905] = scrap_pokemon_details("Enamorus")
pokemon.loc[905, "ability"] = "Cute Charm"
pokemon.loc[905, "hidden_ability"] = "Contrary"

In [32]:
# gen 9 starters
pokemon.loc[906] = scrap_pokemon_details("Sprigatito")
pokemon.loc[907] = scrap_pokemon_details("Floragato")
pokemon.loc[908] = scrap_pokemon_details("Meowscarada")
pokemon.loc[909] = scrap_pokemon_details("Fuecoco")
pokemon.loc[910] = scrap_pokemon_details("Crocalor")
pokemon.loc[911] = scrap_pokemon_details("Skeledirge")
pokemon.loc[912] = scrap_pokemon_details("Quaxly")
pokemon.loc[913] = scrap_pokemon_details("Quaxwell")
pokemon.loc[914] = scrap_pokemon_details("Quaquaval")

In [10]:
#lechonk
pokemon.loc[915] = scrap_pokemon_details("Lechonk")
pokemon.loc[916] = scrap_pokemon_details("Oinkologne")
pokemon.loc[916, "ability"] = "Lingering Aroma / Gluttony"
pokemon.loc[916, "hidden_ability"] = "Thick Fat"
pokemon.loc[917] = scrap_pokemon_details("Tarountula")
pokemon.loc[918] = scrap_pokemon_details("Spidops")
pokemon.loc[919] = scrap_pokemon_details("Nymble")
pokemon.loc[920] = scrap_pokemon_details("Lokix")
pokemon.loc[921] = scrap_pokemon_details("Pawmi")
pokemon.loc[922] = scrap_pokemon_details("Pawmo")
pokemon.loc[923] = scrap_pokemon_details("Pawmot")
pokemon.loc[924] = scrap_pokemon_details("Tandemaus")
pokemon.loc[925] = scrap_pokemon_details("Maushold")
pokemon.loc[926] = scrap_pokemon_details("Fidough")
pokemon.loc[927] = scrap_pokemon_details("Dachsbun")
pokemon.loc[928] = scrap_pokemon_details("Smoliv")
pokemon.loc[929] = scrap_pokemon_details("Dolliv")
pokemon.loc[930] = scrap_pokemon_details("Arboliva")

In [34]:
# nacli
pokemon.loc[931] = scrap_pokemon_details("Squawkabilly")
pokemon.loc[931, "ability"] = "Intimidate / Hustle"
pokemon.loc[931, "hidden_ability"] = "Guts / Sheer Force"
pokemon.loc[932] = scrap_pokemon_details("Nacli")
pokemon.loc[933] = scrap_pokemon_details("Naclstack")
pokemon.loc[934] = scrap_pokemon_details("Garganacl")

In [35]:
# charcadet
pokemon.loc[935] = scrap_pokemon_details("Charcadet")
pokemon.loc[936] = scrap_pokemon_details("Armarouge")
pokemon.loc[937] = scrap_pokemon_details("Ceruledge")
pokemon.loc[938] = scrap_pokemon_details("Tadbulb")
pokemon.loc[939] = scrap_pokemon_details("Bellibolt")
pokemon.loc[940] = scrap_pokemon_details("Wattrel")
pokemon.loc[941] = scrap_pokemon_details("Kilowattrel")
pokemon.loc[942] = scrap_pokemon_details("Maschiff")
pokemon.loc[943] = scrap_pokemon_details("Mabosstiff")
pokemon.loc[944] = scrap_pokemon_details("Shroodle")
pokemon.loc[945] = scrap_pokemon_details("Grafaiai")
pokemon.loc[946] = scrap_pokemon_details("Bramblin")
pokemon.loc[947] = scrap_pokemon_details("Brambleghast")
pokemon.loc[948] = scrap_pokemon_details("Toedscool")
pokemon.loc[949] = scrap_pokemon_details("Toedscruel")
pokemon.loc[950] = scrap_pokemon_details("Klawf")

In [36]:
# capskid
pokemon.loc[951] = scrap_pokemon_details("Capsakid")
pokemon.loc[952] = scrap_pokemon_details("Scovillain")
pokemon.loc[953] = scrap_pokemon_details("Rellor")
pokemon.loc[954] = scrap_pokemon_details("Rabsca")
pokemon.loc[955] = scrap_pokemon_details("Flittle")
pokemon.loc[956] = scrap_pokemon_details("Espathra")
pokemon.loc[957] = scrap_pokemon_details("Tinkatink")
pokemon.loc[958] = scrap_pokemon_details("Tinkatuff")
pokemon.loc[959] = scrap_pokemon_details("Tinkaton")
pokemon.loc[960] = scrap_pokemon_details("Wiglett")
pokemon.loc[961] = scrap_pokemon_details("Wugtrio")
pokemon.loc[962] = scrap_pokemon_details("Bombirdier")

In [37]:
# finizen
pokemon.loc[963] = scrap_pokemon_details("Finizen")
pokemon.loc[964] = scrap_pokemon_details("Palafin")
pokemon.loc[965] = scrap_pokemon_details("Varoom")
pokemon.loc[966] = scrap_pokemon_details("Revavroom")
pokemon.loc[967] = scrap_pokemon_details("Cyclizar")
pokemon.loc[968] = scrap_pokemon_details("Orthworm")
pokemon.loc[969] = scrap_pokemon_details("Glimmet")
pokemon.loc[970] = scrap_pokemon_details("Glimmora")
pokemon.loc[971] = scrap_pokemon_details("Greavard")
pokemon.loc[972] = scrap_pokemon_details("Houndstone")
pokemon.loc[973] = scrap_pokemon_details("Flamigo")
pokemon.loc[974] = scrap_pokemon_details("Cetoddle")
pokemon.loc[975] = scrap_pokemon_details("Cetitan")
pokemon.loc[976] = scrap_pokemon_details("Veluza")
pokemon.loc[977] = scrap_pokemon_details("Dondozo")
pokemon.loc[978] = scrap_pokemon_details("Tatsugiri")

In [38]:
# annihalape
pokemon.loc[979] = scrap_pokemon_details("Annihilape")
pokemon.loc[980] = scrap_pokemon_details("Clodsire")
pokemon.loc[981] = scrap_pokemon_details("Farigiraf")
pokemon.loc[982] = scrap_pokemon_details("Dudunsparce")
pokemon.loc[983] = scrap_pokemon_details("Kingambit")

In [39]:
# 
pokemon.loc[984] = scrap_pokemon_details("Great_Tusk")
pokemon.loc[985] = scrap_pokemon_details("Scream_Tail")
pokemon.loc[986] = scrap_pokemon_details("Brute_Bonnet")
pokemon.loc[987] = scrap_pokemon_details("Flutter_Mane")

In [40]:
pokemon.loc[988] = scrap_pokemon_details("Slither_Wing")
pokemon.loc[989] = scrap_pokemon_details("Sandy_Shocks")
pokemon.loc[990] = scrap_pokemon_details("Iron_Treads")
pokemon.loc[991] = scrap_pokemon_details("Iron_Bundle")
pokemon.loc[992] = scrap_pokemon_details("Iron_Hands")
pokemon.loc[993] = scrap_pokemon_details("Iron_Jugulis")
pokemon.loc[994] = scrap_pokemon_details("Iron_Moth")
pokemon.loc[995] = scrap_pokemon_details("Iron_Thorns")
pokemon.loc[996] = scrap_pokemon_details("Frigibax")
pokemon.loc[997] = scrap_pokemon_details("Arctibax")
pokemon.loc[998] = scrap_pokemon_details("Baxcalibur")

In [41]:
pokemon.loc[999] = scrap_pokemon_details("Gimmighoul")
pokemon.loc[1000] = scrap_pokemon_details("Gholdengo")

In [42]:
pokemon.loc[1001] = scrap_pokemon_details("Wo-Chien")
pokemon.loc[1002] = scrap_pokemon_details("Chien-Pao")
pokemon.loc[1003] = scrap_pokemon_details("Ting-Lu")
pokemon.loc[1004] = scrap_pokemon_details("Chi-Yu")
pokemon.loc[1005] = scrap_pokemon_details("Roaring_Moon")

In [43]:
pokemon.loc[1006] = scrap_pokemon_details("Iron_Valiant")
pokemon.loc[1007] = scrap_pokemon_details("Koraidon")
pokemon.loc[1008] = scrap_pokemon_details("Miraidon")
pokemon.loc[1009] = scrap_pokemon_details("Walking_Wake")
pokemon.loc[1010] = scrap_pokemon_details("Iron_Leaves")

In [14]:
pokemon.loc[1011] = scrap_pokemon_details("Dipplin")
pokemon.loc[1012] = scrap_pokemon_details("Poltchageist")
pokemon.loc[1013] = scrap_pokemon_details("Sinistcha")
pokemon.loc[1014] = scrap_pokemon_details("Okidogi")
pokemon.loc[1015] = scrap_pokemon_details("Munkidori")
pokemon.loc[1016] = scrap_pokemon_details("Fezandipiti")
pokemon.loc[1017] = scrap_pokemon_details("Ogerpon")
pokemon.loc[1017, "ability"] = "Defiant"

In [45]:
pokemon.loc[1018] = scrap_pokemon_details("Archaludon")
pokemon.loc[1019] = scrap_pokemon_details("Hydrapple")
pokemon.loc[1020] = scrap_pokemon_details("Gouging_Fire")
pokemon.loc[1021] = scrap_pokemon_details("Raging_Bolt")
pokemon.loc[1022] = scrap_pokemon_details("Iron_Boulder")
pokemon.loc[1023] = scrap_pokemon_details("Iron_Crown")
pokemon.loc[1024] = scrap_pokemon_details("Terapagos")
pokemon.loc[1025] = scrap_pokemon_details("Pecharunt")

In [61]:
pokemon["hidden_ability"].loc[pokemon["hidden_ability"].isnull() == True] = "None"
pokemon.loc[pokemon["hidden_ability"] == "None"]

,number,name,type,ability,hidden_ability,gender_ratio,hatch_cycle,egg_group,Base_Experience_Yield,height,weight,leveling_rate,color,base_friendship,category,gen
11,#0011,Metapod,Bug,Shed Skin,None,"50% male, 50% female",15cycles,Bug,72,0.7,9.9,Medium Fast,Green,70,Cocoon Pokémon,1
14,#0014,Kakuna,Bug / Poison,Shed Skin,None,"50% male, 50% female",15cycles,Bug,72,0.6,10,Medium Fast,Yellow,70,Cocoon Pokémon,1
92,#0092,Gastly,Ghost / Poison,Levitate,None,"50% male, 50% female",20cycles,Amorphous,62,1.3,0.1,Medium Slow,Purple,70,Gas Pokémon,1
93,#0093,Haunter,Ghost / Poison,Levitate,None,"50% male, 50% female",20cycles,Amorphous,142,1.6,0.1,Medium Slow,Purple,70,Gas Pokémon,1
151,#0151,Mew,Psychic,Synchronize,None,Gender unknown,120cycles,No Eggs Discovered,270,0.4,4,Medium Slow,Pink,100,New Species Pokémon,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,#1020,Gouging Fire,Fire / Dragon,Protosynthesis,None,Gender unknown,50cycles,No Eggs Discovered,295,3.5,590,Slow,Brown,0,Paradox Pokémon,9
1021,#1021,Raging Bolt,Electric / Dragon,Protosynthesis,None,Gender unknown,50cycles,No Eggs Discovered,295,5.2,480,Slow,Yellow,0,Paradox Pokémon,9
1022,#1022,Iron Boulder,Rock / Psychic,Quark Drive,None,Gender unknown,50cycles,No Eggs Discovered,295,1.5,162.5,Slow,Gray,0,Paradox Pokémon,9
1023,#1023,Iron Crown,Steel / Psychic,Quark Drive,None,Gender unknown,50cycles,No Eggs Discovered,295,1.6,156,Slow,Blue,0,Paradox Pokémon,9


In [28]:
# applying gen 
pokemon.loc[pokemon.index <= 151, "gen"] = "1" 
pokemon.loc[(pokemon.index > 151) & (pokemon.index <= 251), "gen"] = "2"
pokemon.loc[(pokemon.index > 251) & (pokemon.index <= 386), "gen"] = "3"
pokemon.loc[(pokemon.index > 386) & (pokemon.index <= 493), "gen"] = "4"
pokemon.loc[(pokemon.index > 493) & (pokemon.index <= 649), "gen"] = "5"
pokemon.loc[(pokemon.index > 649) & (pokemon.index <= 721), "gen"] = "6"
pokemon.loc[(pokemon.index > 721) & (pokemon.index <= 809), "gen"] = "7"
pokemon.loc[(pokemon.index > 809) & (pokemon.index <= 905), "gen"] = "8"
pokemon.loc[(pokemon.index > 905) & (pokemon.index <= 1025), "gen"] = "9"

In [11]:
pokemon["color"] = pokemon["color"].str.replace(" ", "")
pokemon["egg_group"] = pokemon["egg_group"].str.replace("and ", " / ")

In [29]:
pokemon.to_csv("/home/user/Desktop/Data_analysis_projects/Pokemon/data.csv")
pokemon[0:50]

,number,name,type,ability,hidden_ability,gender_ratio,hatch_cycle,egg_group,Base_Experience_Yield,height,weight,leveling_rate,color,base_friendship,category,gen
1,#0001,Bulbasaur,Grass / Poison,Overgrow,Chlorophyll,"87.5% male, 12.5% female",20cycles,Monster / Grass,64,0.7,6.9,Medium Slow,Green,70,Seed Pokémon,1
2,#0002,Ivysaur,Grass / Poison,Overgrow,Chlorophyll,"87.5% male, 12.5% female",20cycles,Monster / Grass,142,1,13,Medium Slow,Green,70,Seed Pokémon,1
3,#0003,Venusaur,Grass / Poison,Overgrow / Chlorophyll,Thick Fat,"87.5% male, 12.5% female",20cycles,Monster / Grass,236,2,100,Medium Slow,Green,70,Seed Pokémon,1
4,#0004,Charmander,Fire,Blaze,Solar Power,"87.5% male, 12.5% female",20cycles,Monster / Dragon,62,0.6,8.5,Medium Slow,Red,70,Lizard Pokémon,1
5,#0005,Charmeleon,Fire,Blaze,Solar Power,"87.5% male, 12.5% female",20cycles,Monster / Dragon,142,1.1,19,Medium Slow,Red,70,Flame Pokémon,1
6,#0006,Charizard,Fire / Flying,Blaze,Solar Power,"87.5% male, 12.5% female",20cycles,Monster / Dragon,240,1.7,90.5,Medium Slow,Red,70,Flame Pokémon,1
7,#0007,Squirtle,Water,Torrent,Rain Dish,"87.5% male, 12.5% female",20cycles,Monster / Water 1,63,0.5,9,Medium Slow,Blue,70,Tiny Turtle Pokémon,1
8,#0008,Wartortle,Water,Torrent,Rain Dish,"87.5% male, 12.5% female",20cycles,Monster / Water 1,142,1,22.5,Medium Slow,Blue,70,Turtle Pokémon,1
9,#0009,Blastoise,Water,Torrent / Rain Dish,Mega Launcher,"87.5% male, 12.5% female",20cycles,Monster / Water 1,239,1.6,85.5,Medium Slow,Blue,70,Shellfish Pokémon,1
10,#0010,Caterpie,Bug,Shield Dust,Run Away,"50% male, 50% female",15cycles,Bug,39,0.3,2.9,Medium Fast,Green,70,Worm Pokémon,1


In [ ]:
text = requests.get("https://bulbapedia.bulbagarden.net/wiki/Wigglytuff_(Pok%C3%A9mon)").text
soup = BeautifulSoup(text, "lxml")
data = soup.find_all("td", class_ = "roundy")

In [ ]:
# ability and hidden ability
ability_data = data[2]
ability_data2 = ability_data.find_all(["a", "span"])
clear_ability = []
junk_text = ["Cacophony", "Abilities", "Gen IV+", "Battle Bond", "Battle Bond Greninja", "Gen VI+"]

for i in ability_data2:
    text = i.text.strip()
    if text and text not in junk_text:
        clear_ability.append(text)
        
if len(clear_ability) > 5:
    ability = clear_ability[0], clear_ability[2]
    hidden_ability = clear_ability[4]
elif len(clear_ability) < 5:
    ability = clear_ability[0]
    hidden_ability = clear_ability[2]
print(len(clear_ability), ability, hidden_ability)
clear_ability

In [59]:
pokemon.loc[25]

number                                  #0025
name                                  Pikachu
type                                 Electric
ability                                Static
hidden_ability                  Lightning Rod
gender_ratio             50% male, 50% female
hatch_cycle                          10cycles
egg_group                       Field / Fairy
Base_Experience_Yield                     112
height                                    0.4
weight                                    6.0
leveling_rate                     Medium Fast
color                                  Yellow
base_friendship                            70
category                        Mouse Pokémon
gen                                         1
Name: 25, dtype: object